In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP_XGB/visualizations_mlp/COP'
        self.xgb_viz_dir = 'OpenAI_MLP_XGB/visualizations_xgb/COP'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs(self.xgb_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP_XGB/visualizations_summary/COP', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def get_xgb_parameters(self):
        """
        Get XGBoost parameters and parameter tuning space optimized for high-dimensional embeddings.
        Uses a single parameter set for both Title and Full text embeddings since they have the same dimension (1536).
        
        Returns:
            Dictionary of base parameters and parameter grid for tuning
        """
        # Setup XGBoost base parameters optimized for high-dimensional embeddings
        base_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'n_estimators': 100,
            'max_depth': 3,
            'learning_rate': 0.03,
            'subsample': 0.7,          # Row subsampling to prevent overfitting
            'colsample_bytree': 0.7,   # Column subsampling to handle high dimensionality
            'min_child_weight': 3,     # Prevents overfitting on high-dimensional embeddings
            'reg_alpha': 0.5,          # L1 regularization
            'reg_lambda': 1.0,         # L2 regularization
            'random_state': 42
        }
        
        # Parameter grid for tuning
        param_grid = {
            'n_estimators': [50, 100, 150],
            'max_depth': [2, 3, 4],
            'learning_rate': [0.01, 0.03, 0.05],
            'min_child_weight': [2, 3, 5],
            'subsample': [0.6, 0.7, 0.8],
            'colsample_bytree': [0.6, 0.7, 0.8],
            'reg_alpha': [0.1, 0.5, 1.0],
            'reg_lambda': [1.0, 2.0, 3.0]
        }
        
        return base_params, param_grid
    
    def create_xgb_model(self):
        """
        Create an XGBoost model for document-level embeddings.
        
        Returns:
            XGBoost model
        """
        # Get base parameters
        base_params, _ = self.get_xgb_parameters()
        
        # Create XGBoost model with additional required parameters
        base_params['use_label_encoder'] = False  # Avoid deprecation warning
        base_params['eval_metric'] = 'auc'        # Explicitly set eval metric
        
        model = xgb.XGBClassifier(**base_params)
        
        print(f"Created XGBoost model")
        return model
    
    def train_xgb_model(self, model, X_train, y_train, X_val, y_val, early_stopping_rounds=10):
        """
        Train XGBoost model with early stopping using scikit-learn API.
        
        Args:
            model: XGBoost model
            X_train, y_train: Training data
            X_val, y_val: Validation data
            early_stopping_rounds: Early stopping patience
        
        Returns:
            Trained model, training history
        """
        # Set up evaluation set for early stopping
        eval_set = [(X_train, y_train), (X_val, y_val)]
        
        # Train the model with the scikit-learn interface
        model.fit(
            X_train, y_train,
            eval_set=eval_set,
            eval_metric='auc',
            early_stopping_rounds=early_stopping_rounds,
            verbose=True
        )
    
        # Get the evaluation results
        evals_result = {
            'train': {'error': [], 'auc': []},
            'validation': {'error': [], 'auc': []}
        }
    
        # Extract evaluation results if available
        if hasattr(model, 'evals_result'):
            results = model.evals_result()
            if results:
                # Map the results to our expected format
                for i, metric in enumerate(['error', 'auc']):
                    if f'validation_{i}' in results:
                        validation_key = f'validation_{i}'
                        if 'auc' in results[validation_key]:
                            evals_result['validation']['auc'] = results[validation_key]['auc']
                        if 'error' in results[validation_key]:
                            evals_result['validation']['error'] = results[validation_key]['error']
                
                    if f'train_{i}' in results:
                        train_key = f'train_{i}'
                        if 'auc' in results[train_key]:
                            evals_result['train']['auc'] = results[train_key]['auc']
                        if 'error' in results[train_key]:
                            evals_result['train']['error'] = results[train_key]['error']
    
        return model, evals_result
    
    def plot_xgb_learning_curves(self, evals_result, text_col, label_col, layer_num):
        """
        Plot XGBoost final learning curves.
        
        Args:
            evals_result: Results from training
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        
        # Plot training error
        plt.plot(evals_result['train']['error'], label='Training Error', color='blue')
        plt.plot(evals_result['validation']['error'], label='Validation Error', color='orange')
        plt.title(f'XGBoost Error Curves ({display_text}, {label_col}, Layer {layer_num})')
        plt.xlabel('Boosting Rounds')
        plt.ylabel('Error')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        
        plt.subplot(1, 2, 2)
        
        # Plot AUC
        plt.plot(evals_result['train']['auc'], label='Training AUC', color='blue')
        plt.plot(evals_result['validation']['auc'], label='Validation AUC', color='orange')
        plt.title(f'XGBoost AUC Curves ({display_text}, {label_col}, Layer {layer_num})')
        plt.xlabel('Boosting Rounds')
        plt.ylabel('AUC')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        
        # Save plot
        plt.savefig(f"{self.xgb_viz_dir}/xgb_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png")
        plt.close()
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP' or 'XGBoost')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            else:  # XGBoost
                # XGBoost model training
                model = self.create_xgb_model()
                print(f"Training XGBoost model...")
                
                # Train model with early stopping
                model, evals_result = self.train_xgb_model(
                    model,
                    X_train, y_train,
                    X_val, y_val,
                    early_stopping_rounds=10
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Plot learning curves
                self.plot_xgb_learning_curves(evals_result, text_col, label_col, i+1)
                
                # Store training history
                training_history = evals_result
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP', 'XGBoost']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        xgb_data = df[df['Model'] == 'XGBoost']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/COP', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # 2. Performance comparison for XGBoost
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for XGBoost
        x = np.arange(len(xgb_data))
        
        plt.bar(x - width/2, xgb_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, xgb_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('XGBoost Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of XGBoost Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in xgb_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(xgb_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(xgb_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(xgb_data['Avg Accuracy'].max(), xgb_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/COP', "xgb_performance_comparison.png")
        plt.savefig(save_path)
        print(f"XGBoost summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP and XGBoost models."""
        # Prepare data for visualization
        mlp_layer_data = []
        xgb_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
                else:  # XGBoost
                    xgb_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
        self._create_model_layer_visualization(xgb_layer_data, 'XGBoost')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/COP', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP_XGB/visualizations_mlp/COP', 'OpenAI_MLP_XGB/visualizations_xgb/COP', 
                      'OpenAI_MLP_XGB/visualizations_summary/COP']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_semantics_SP500_database/wall_street_news_semantics_COP_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP_XGB' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {0: 431, 1: 407}
Class distribution for long-term prediction: {1: 440, 0: 398}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_84"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 dense_316 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_231 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_170 (Dropout)       (None, 512)               0         
                                                                 
 dense_317 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_232 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_171 (Dropout)       (None, 256)               0         
                                                                 
 dense_318 (Dense)           (None, 128)               32896     
          

21/21 [==============================] - 0s 11ms/step - loss: 0.4135 - accuracy: 0.7943 - val_loss: 0.6873 - val_accuracy: 0.5347
Epoch 17/100
16/21 [=====================>........] - ETA: 0s - loss: 0.3901 - accuracy: 0.8574
Epoch 17: val_loss did not improve from 0.68725
21/21 [==============================] - 0s 12ms/step - loss: 0.3786 - accuracy: 0.8589 - val_loss: 0.6892 - val_accuracy: 0.5248
Epoch 18/100
19/21 [==========================>...] - ETA: 0s - loss: 0.3575 - accuracy: 0.8503
Epoch 18: val_loss did not improve from 0.68725
21/21 [==============================] - 0s 11ms/step - loss: 0.3561 - accuracy: 0.8514 - val_loss: 0.6924 - val_accuracy: 0.4950
Epoch 19/100
20/21 [===========================>..] - ETA: 0s - loss: 0.3649 - accuracy: 0.8500
Epoch 19: val_loss did not improve from 0.68725
21/21 [==============================] - 0s 13ms/step - loss: 0.3668 - accuracy: 0.8514 - val_loss: 0.6904 - val_accuracy: 0.5050
Epoch 20/100
21/21 [============================

13/13 [==============================] - 0s 19ms/step - loss: 0.5879 - accuracy: 0.6783 - val_loss: 0.6597 - val_accuracy: 0.6718
Epoch 7/100
11/13 [========================>.....] - ETA: 0s - loss: 0.5964 - accuracy: 0.6903
Epoch 7: val_loss improved from 0.65974 to 0.65949, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_title_L_label_layer_1.weights.h5
13/13 [==============================] - 0s 18ms/step - loss: 0.5772 - accuracy: 0.7032 - val_loss: 0.6595 - val_accuracy: 0.6718
Epoch 8/100
11/13 [========================>.....] - ETA: 0s - loss: 0.5656 - accuracy: 0.6903
Epoch 8: val_loss did not improve from 0.65949
13/13 [==============================] - 0s 15ms/step - loss: 0.5631 - accuracy: 0.6833 - val_loss: 0.6596 - val_accuracy: 0.6718
Epoch 9/100
11/13 [========================>.....] - ETA: 0s - loss: 0.5368 - accuracy: 0.7131
Epoch 9: val_loss did not improve from 0.65949
13/13 [==============================] - 0s 14ms/step - loss: 0.5244 - accuracy: 0.

Epoch 4/100
17/17 [==============================] - ETA: 0s - loss: 0.7335 - accuracy: 0.6128
Epoch 4: val_loss improved from 0.63323 to 0.62323, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_title_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 15ms/step - loss: 0.7335 - accuracy: 0.6128 - val_loss: 0.6232 - val_accuracy: 0.7368
Epoch 5/100
15/17 [=========================>....] - ETA: 0s - loss: 0.6477 - accuracy: 0.6479
Epoch 5: val_loss improved from 0.62323 to 0.61691, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_title_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 16ms/step - loss: 0.6447 - accuracy: 0.6410 - val_loss: 0.6169 - val_accuracy: 0.7368
Epoch 6/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6223 - accuracy: 0.6680
Epoch 6: val_loss improved from 0.61691 to 0.61412, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_title_L_label_layer_2.weights.h5
17/17 [====

...

16/17 [===========================>..] - ETA: 0s - loss: 0.7442 - accuracy: 0.5879
Epoch 3: val_loss improved from 0.69260 to 0.69246, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_full_text_S_label_layer_2.weights.h5
17/17 [==============================] - 0s 16ms/step - loss: 0.7526 - accuracy: 0.5808 - val_loss: 0.6925 - val_accuracy: 0.5338
Epoch 4/100
15/17 [=========================>....] - ETA: 0s - loss: 0.6960 - accuracy: 0.6062
Epoch 4: val_loss did not improve from 0.69246
17/17 [==============================] - 0s 14ms/step - loss: 0.6918 - accuracy: 0.6071 - val_loss: 0.6927 - val_accuracy: 0.5338
Epoch 5/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6602 - accuracy: 0.6191
Epoch 5: val_loss did not improve from 0.69246
17/17 [==============================] - 0s 14ms/step - loss: 0.6577 - accuracy: 0.6184 - val_loss: 0.6933 - val_accuracy: 0.5338
Epoch 6/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6188 - accuracy: 0.6

13/13 [==============================] - 0s 18ms/step - loss: 0.6520 - accuracy: 0.6584 - val_loss: 0.6641 - val_accuracy: 0.6718
Epoch 7/100
11/13 [========================>.....] - ETA: 0s - loss: 0.5747 - accuracy: 0.6932
Epoch 7: val_loss improved from 0.66411 to 0.66145, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_full_text_L_label_layer_1.weights.h5
13/13 [==============================] - 0s 17ms/step - loss: 0.5760 - accuracy: 0.6883 - val_loss: 0.6615 - val_accuracy: 0.6718
Epoch 8/100
11/13 [========================>.....] - ETA: 0s - loss: 0.5341 - accuracy: 0.7244
Epoch 8: val_loss improved from 0.66145 to 0.66090, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_full_text_L_label_layer_1.weights.h5
13/13 [==============================] - 0s 18ms/step - loss: 0.5425 - accuracy: 0.7207 - val_loss: 0.6609 - val_accuracy: 0.6718
Epoch 9/100
11/13 [========================>.....] - ETA: 0s - loss: 0.5482 - accuracy: 0.7045
Epoch 9: val_loss imp

                                                                 
 dense_355 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
16/17 [===========================>..] - ETA: 0s - loss: 0.8415 - accuracy: 0.5547
Epoch 1: val_loss improved from inf to 0.71341, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_full_text_L_label_layer_2.weights.h5
17/17 [==============================] - 2s 25ms/step - loss: 0.8436 - accuracy: 0.5508 - val_loss: 0.7134 - val_accuracy: 0.2632
Epoch 2/100
16/17 [===========================>..] - ETA: 0s - loss: 0.8196 - accuracy: 0.5684
Epoch 2: val_loss did not improve from 0.71341
17/17 [==============================] - 0s 13ms/step - loss: 0.8145 - accuracy: 0.5733 - val_lo

21/21 [==============================] - 0s 11ms/step - loss: 0.6974 - accuracy: 0.6156 - val_loss: 0.6968 - val_accuracy: 0.4950
Epoch 4/100
21/21 [==============================] - ETA: 0s - loss: 0.6715 - accuracy: 0.6201
Epoch 4: val_loss did not improve from 0.69400
21/21 [==============================] - 0s 12ms/step - loss: 0.6715 - accuracy: 0.6201 - val_loss: 0.6987 - val_accuracy: 0.4950
Epoch 5/100
18/21 [========================>.....] - ETA: 0s - loss: 0.6329 - accuracy: 0.6510
Epoch 5: val_loss did not improve from 0.69400
21/21 [==============================] - 0s 11ms/step - loss: 0.6252 - accuracy: 0.6652 - val_loss: 0.6998 - val_accuracy: 0.4950
Epoch 6/100
19/21 [==========================>...] - ETA: 0s - loss: 0.6450 - accuracy: 0.6349
Epoch 6: val_loss did not improve from 0.69400
21/21 [==============================] - 0s 11ms/step - loss: 0.6455 - accuracy: 0.6381 - val_loss: 0.7005 - val_accuracy: 0.4950
Epoch 7/100
21/21 [==============================] - E

TypeError: XGBClassifier.fit() got an unexpected keyword argument 'eval_metric'

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP_XGB/visualizations_mlp/COP'
        self.xgb_viz_dir = 'OpenAI_MLP_XGB/visualizations_xgb/COP'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs(self.xgb_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP_XGB/visualizations_summary/COP', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def get_xgb_parameters(self):
        """
        Get XGBoost parameters optimized for high-dimensional embeddings.
        Uses a single parameter set for both Title and Full text embeddings since they have the same dimension (1536).
        
        Returns:
            Dictionary of base parameters
        """
        # Setup XGBoost base parameters optimized for high-dimensional embeddings
        base_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'n_estimators': 100,
            'max_depth': 3,
            'learning_rate': 0.05,
            'subsample': 0.7,          # Row subsampling to prevent overfitting
            'colsample_bytree': 0.5,   # Column subsampling to handle high dimensionality
            'min_child_weight': 3,     # Prevents overfitting on high-dimensional embeddings
            'reg_alpha': 1.0,          # L1 regularization
            'reg_lambda': 2.0,         # L2 regularization
            'random_state': 42,
            'use_label_encoder': False # Avoid deprecation warning
        }
        
        return base_params
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP' or 'XGBoost')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            else:  # XGBoost - simplified approach to fix API issues
                # Get XGBoost parameters
                base_params = self.get_xgb_parameters()
                
                # Create and train XGBoost model
                print(f"Creating and training XGBoost model...")
                model = xgb.XGBClassifier(**base_params)
                
                # Only use validation set for evaluation (not training set)
                eval_set = [(X_val, y_val)]
                
                # Train the model
                model.fit(
                    X_train, y_train,
                    eval_set=eval_set,
                    verbose=True
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create a simple summary for XGBoost (no detailed learning curves available)
                plt.figure(figsize=(10, 6))
                plt.text(0.5, 0.5, f'XGBoost Model Trained Successfully\nAccuracy: {accuracy_score(y_test, y_pred):.4f}\nAUC: {roc_auc_score(y_test, y_pred_proba):.4f}',
                         ha='center', va='center', size=14, fontweight='bold')
                plt.title(f'XGBoost Results ({display_text}, {label_col}, Layer {i+1})')
                plt.axis('off')
                plt.tight_layout()
                plt.savefig(f"{visualization_dir}/xgb_{display_text.replace(' ', '_')}_{label_col}_layer_{i+1}.png")
                plt.close()
                
                # Use None for training history since detailed learning curves aren't available
                training_history = None
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP', 'XGBoost']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        xgb_data = df[df['Model'] == 'XGBoost']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/COP', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # 2. Performance comparison for XGBoost
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for XGBoost
        x = np.arange(len(xgb_data))
        
        plt.bar(x - width/2, xgb_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, xgb_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('XGBoost Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of XGBoost Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in xgb_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(xgb_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(xgb_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(xgb_data['Avg Accuracy'].max(), xgb_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/COP', "xgb_performance_comparison.png")
        plt.savefig(save_path)
        print(f"XGBoost summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP and XGBoost models."""
        # Prepare data for visualization
        mlp_layer_data = []
        xgb_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
                else:  # XGBoost
                    xgb_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
        self._create_model_layer_visualization(xgb_layer_data, 'XGBoost')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/COP', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP_XGB/visualizations_mlp/COP', 'OpenAI_MLP_XGB/visualizations_xgb/COP', 
                      'OpenAI_MLP_XGB/visualizations_summary/COP']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_semantics_SP500_database/wall_street_news_semantics_COP_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP_XGB' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {0: 431, 1: 407}
Class distribution for long-term prediction: {1: 440, 0: 398}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_121"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_464 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_342 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_244 (Dropout)       (None, 512)               0         
                                                                 
 dense_465 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_343 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_245 (Dropout)       (None, 256)               0         
          

 batch_normalization_347 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_471 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
20/21 [===========================>..] - ETA: 0s - loss: 0.8847 - accuracy: 0.4906
Epoch 1: val_loss improved from inf to 0.68800, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_title_S_label_layer_3.weights.h5
21/21 [==============================] - 2s 23ms/step - loss: 0.8884 - accuracy: 0.4910 - val_loss: 0.6880 - val_accuracy: 0.5545
Epoch 2/100
21/21 [==============================] - ETA: 0s - loss: 0.8133 - accuracy: 0.5210
Epoch 2: val_lo

Training MLP model...
Epoch 1/100
10/13 [======================>.......] - ETA: 0s - loss: 0.8752 - accuracy: 0.4969
Epoch 1: val_loss improved from inf to 0.72254, saving model to OpenAI_MLP_XGB/visualizations_mlp/COP\best_mlp_title_L_label_layer_1.weights.h5
13/13 [==============================] - 2s 30ms/step - loss: 0.8811 - accuracy: 0.4838 - val_loss: 0.7225 - val_accuracy: 0.3282
Epoch 2/100
10/13 [======================>.......] - ETA: 0s - loss: 0.7732 - accuracy: 0.5562
Epoch 2: val_loss did not improve from 0.72254
13/13 [==============================] - 0s 15ms/step - loss: 0.7860 - accuracy: 0.5586 - val_loss: 0.7506 - val_accuracy: 0.3282
Epoch 3/100
11/13 [========================>.....] - ETA: 0s - loss: 0.7570 - accuracy: 0.5568
Epoch 3: val_loss did not improve from 0.72254
13/13 [==============================] - 0s 15ms/step - loss: 0.7689 - accuracy: 0.5511 - val_loss: 0.7804 - val_accuracy: 0.3282
Epoch 4/100
11/13 [========================>.....] - ETA: 0s - lo

16/17 [===========================>..] - ETA: 0s - loss: 0.6676 - accuracy: 0.6504
Epoch 5: val_loss did not improve from 0.71472
17/17 [==============================] - 0s 14ms/step - loss: 0.6654 - accuracy: 0.6523 - val_loss: 0.7339 - val_accuracy: 0.2632
Epoch 6/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6072 - accuracy: 0.6641
Epoch 6: val_loss did not improve from 0.71472
17/17 [==============================] - 0s 14ms/step - loss: 0.6071 - accuracy: 0.6617 - val_loss: 0.7311 - val_accuracy: 0.2632
Epoch 7/100
15/17 [=========================>....] - ETA: 0s - loss: 0.5689 - accuracy: 0.6896
Epoch 7: val_loss did not improve from 0.71472
17/17 [==============================] - 0s 14ms/step - loss: 0.5833 - accuracy: 0.6786 - val_loss: 0.7312 - val_accuracy: 0.2632
Epoch 8/100
16/17 [===========================>..] - ETA: 0s - loss: 0.5414 - accuracy: 0.7148
Epoch 8: val_loss did not improve from 0.71472
17/17 [==============================] - 0s 13ms/step 

...

21/21 [==============================] - 2s 23ms/step - loss: 0.8551 - accuracy: 0.4895 - val_loss: 0.6951 - val_accuracy: 0.4950
Epoch 2/100
17/21 [=======================>......] - ETA: 0s - loss: 0.7751 - accuracy: 0.5717
Epoch 2: val_loss did not improve from 0.69511
21/21 [==============================] - 0s 14ms/step - loss: 0.7722 - accuracy: 0.5826 - val_loss: 0.6969 - val_accuracy: 0.4950
Epoch 3/100
21/21 [==============================] - ETA: 0s - loss: 0.7433 - accuracy: 0.5766
Epoch 3: val_loss did not improve from 0.69511
21/21 [==============================] - 0s 12ms/step - loss: 0.7433 - accuracy: 0.5766 - val_loss: 0.6985 - val_accuracy: 0.4950
Epoch 4/100
19/21 [==========================>...] - ETA: 0s - loss: 0.6630 - accuracy: 0.6530
Epoch 4: val_loss did not improve from 0.69511
21/21 [==============================] - 0s 11ms/step - loss: 0.6695 - accuracy: 0.6456 - val_loss: 0.6996 - val_accuracy: 0.4950
Epoch 5/100
21/21 [==============================] - E

Training data: 532 samples
Validation data: 133 samples
Test data: 101 samples
Embeddings shapes - Train: (532, 1536), Val: (133, 1536), Test: (101, 1536)
Creating and training XGBoost model...
[0]	validation_0-auc:0.49523
[1]	validation_0-auc:0.38823
[2]	validation_0-auc:0.38130
[3]	validation_0-auc:0.39539
[4]	validation_0-auc:0.40572
[5]	validation_0-auc:0.46365
[6]	validation_0-auc:0.46615
[7]	validation_0-auc:0.46729
[8]	validation_0-auc:0.47388
[9]	validation_0-auc:0.48228
[10]	validation_0-auc:0.47478
[11]	validation_0-auc:0.49364
[12]	validation_0-auc:0.49364
[13]	validation_0-auc:0.50931
[14]	validation_0-auc:0.49500
[15]	validation_0-auc:0.49114
[16]	validation_0-auc:0.48523
[17]	validation_0-auc:0.49409
[18]	validation_0-auc:0.49886
[19]	validation_0-auc:0.49977
[20]	validation_0-auc:0.50477
[21]	validation_0-auc:0.50023
[22]	validation_0-auc:0.51090
[23]	validation_0-auc:0.51340
[24]	validation_0-auc:0.51045
[25]	validation_0-auc:0.51386
[26]	validation_0-auc:0.52272
[27]	v

[79]	validation_0-auc:0.54941
[80]	validation_0-auc:0.54392
[81]	validation_0-auc:0.54980
[82]	validation_0-auc:0.55686
[83]	validation_0-auc:0.55882
[84]	validation_0-auc:0.55647
[85]	validation_0-auc:0.55216
[86]	validation_0-auc:0.54392
[87]	validation_0-auc:0.54275
[88]	validation_0-auc:0.54235
[89]	validation_0-auc:0.54863
[90]	validation_0-auc:0.54902
[91]	validation_0-auc:0.54627
[92]	validation_0-auc:0.54314
[93]	validation_0-auc:0.54627
[94]	validation_0-auc:0.54549
[95]	validation_0-auc:0.54627
[96]	validation_0-auc:0.54980
[97]	validation_0-auc:0.54588
[98]	validation_0-auc:0.54627
[99]	validation_0-auc:0.54039
Test Accuracy for Layer 3: 0.5286
Test AUC for Layer 3: 0.5964

Average Test Accuracy across all layers: 0.4970
Average Test AUC across all layers: 0.5443

Training XGBoost model for Full text and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Vali

[4]	validation_0-auc:0.39861
[5]	validation_0-auc:0.43591
[6]	validation_0-auc:0.47024
[7]	validation_0-auc:0.45437
[8]	validation_0-auc:0.43135
[9]	validation_0-auc:0.44802
[10]	validation_0-auc:0.44683
[11]	validation_0-auc:0.43333
[12]	validation_0-auc:0.42500
[13]	validation_0-auc:0.42540
[14]	validation_0-auc:0.42183
[15]	validation_0-auc:0.41270
[16]	validation_0-auc:0.39683
[17]	validation_0-auc:0.37381
[18]	validation_0-auc:0.39048
[19]	validation_0-auc:0.39683
[20]	validation_0-auc:0.39683
[21]	validation_0-auc:0.38452
[22]	validation_0-auc:0.41270
[23]	validation_0-auc:0.41032
[24]	validation_0-auc:0.42500
[25]	validation_0-auc:0.41667
[26]	validation_0-auc:0.42103
[27]	validation_0-auc:0.42222
[28]	validation_0-auc:0.41310
[29]	validation_0-auc:0.40317
[30]	validation_0-auc:0.41865
[31]	validation_0-auc:0.42063
[32]	validation_0-auc:0.40992
[33]	validation_0-auc:0.42619
[34]	validation_0-auc:0.42024
[35]	validation_0-auc:0.41587
[36]	validation_0-auc:0.41230
[37]	validation_

[42]	validation_0-auc:0.55889
[43]	validation_0-auc:0.55160
[44]	validation_0-auc:0.54548
[45]	validation_0-auc:0.55306
[46]	validation_0-auc:0.56618
[47]	validation_0-auc:0.56939
[48]	validation_0-auc:0.56297
[49]	validation_0-auc:0.55656
[50]	validation_0-auc:0.56443
[51]	validation_0-auc:0.57172
[52]	validation_0-auc:0.56618
[53]	validation_0-auc:0.56997
[54]	validation_0-auc:0.57172
[55]	validation_0-auc:0.57289
[56]	validation_0-auc:0.57493
[57]	validation_0-auc:0.57201
[58]	validation_0-auc:0.55948
[59]	validation_0-auc:0.55773
[60]	validation_0-auc:0.55015
[61]	validation_0-auc:0.54898
[62]	validation_0-auc:0.56385
[63]	validation_0-auc:0.56327
[64]	validation_0-auc:0.56181
[65]	validation_0-auc:0.56414
[66]	validation_0-auc:0.56589
[67]	validation_0-auc:0.55918
[68]	validation_0-auc:0.56356
[69]	validation_0-auc:0.56210
[70]	validation_0-auc:0.56093
[71]	validation_0-auc:0.56035
[72]	validation_0-auc:0.56006
[73]	validation_0-auc:0.55802
[74]	validation_0-auc:0.55918
[75]	valid

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP_XGB/visualizations_mlp/CVX'
        self.xgb_viz_dir = 'OpenAI_MLP_XGB/visualizations_xgb/CVX'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs(self.xgb_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP_XGB/visualizations_summary/CVX', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def get_xgb_parameters(self):
        """
        Get XGBoost parameters optimized for high-dimensional embeddings.
        Uses a single parameter set for both Title and Full text embeddings since they have the same dimension (1536).
        
        Returns:
            Dictionary of base parameters
        """
        # Setup XGBoost base parameters optimized for high-dimensional embeddings
        base_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'n_estimators': 80,
            'max_depth': 3,
            'learning_rate': 0.001,
            'subsample': 0.7,          # Row subsampling to prevent overfitting
            'colsample_bytree': 0.5,   # Column subsampling to handle high dimensionality
            'min_child_weight': 3,     # Prevents overfitting on high-dimensional embeddings
            'reg_alpha': 1.0,          # L1 regularization
            'reg_lambda': 2.0,         # L2 regularization
            'random_state': 42,
            'use_label_encoder': False # Avoid deprecation warning
        }
        
        return base_params
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP' or 'XGBoost')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            else:  # XGBoost - simplified approach to fix API issues
                # Get XGBoost parameters
                base_params = self.get_xgb_parameters()
                
                # Create and train XGBoost model
                print(f"Creating and training XGBoost model...")
                model = xgb.XGBClassifier(**base_params)
                
                # Only use validation set for evaluation (not training set)
                eval_set = [(X_val, y_val)]
                
                # Train the model
                model.fit(
                    X_train, y_train,
                    eval_set=eval_set,
                    verbose=True
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create a simple summary for XGBoost (no detailed learning curves available)
                plt.figure(figsize=(10, 6))
                plt.text(0.5, 0.5, f'XGBoost Model Trained Successfully\nAccuracy: {accuracy_score(y_test, y_pred):.4f}\nAUC: {roc_auc_score(y_test, y_pred_proba):.4f}',
                         ha='center', va='center', size=14, fontweight='bold')
                plt.title(f'XGBoost Results ({display_text}, {label_col}, Layer {i+1})')
                plt.axis('off')
                plt.tight_layout()
                plt.savefig(f"{visualization_dir}/xgb_{display_text.replace(' ', '_')}_{label_col}_layer_{i+1}.png")
                plt.close()
                
                # Use None for training history since detailed learning curves aren't available
                training_history = None
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP', 'XGBoost']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        xgb_data = df[df['Model'] == 'XGBoost']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/CVX', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # 2. Performance comparison for XGBoost
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for XGBoost
        x = np.arange(len(xgb_data))
        
        plt.bar(x - width/2, xgb_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, xgb_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('XGBoost Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of XGBoost Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in xgb_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(xgb_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(xgb_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(xgb_data['Avg Accuracy'].max(), xgb_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/CVX', "xgb_performance_comparison.png")
        plt.savefig(save_path)
        print(f"XGBoost summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP and XGBoost models."""
        # Prepare data for visualization
        mlp_layer_data = []
        xgb_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
                else:  # XGBoost
                    xgb_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
        self._create_model_layer_visualization(xgb_layer_data, 'XGBoost')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/CVX', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP_XGB/visualizations_mlp/CVX', 'OpenAI_MLP_XGB/visualizations_xgb/CVX', 
                      'OpenAI_MLP_XGB/visualizations_summary/CVX']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_semantics_SP500_database/wall_street_news_semantics_CVX_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP_XGB' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {1: 452, 0: 386}
Class distribution for long-term prediction: {1: 464, 0: 374}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_133"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   

11/13 [========================>.....] - ETA: 0s - loss: 0.3501 - accuracy: 0.8580Restoring model weights from the end of the best epoch: 4.

Epoch 19: val_loss did not improve from 0.68468
13/13 [==============================] - 0s 15ms/step - loss: 0.3353 - accuracy: 0.8653 - val_loss: 0.7055 - val_accuracy: 0.5649
Epoch 19: early stopping
5/5 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_Title_S_label_layer_1.png
Test Accuracy for Layer 1: 0.6090
Test AUC for Layer 1: 0.6256

Layer 2:
Training period: 01/11/2014 - 31/10/2021
Validation period: 01/11/2021 - 31/10/2022
Testing period: 01/11/2022 - 31/10/2023
Training data: 532 samples
Validation data: 133 samples
Test data: 101 samples
Embeddings shapes - Train: (532, 1536), Val: (133, 1536), Test: (101, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_134"
_________________________________________________________________
 L

 batch_normalization_381 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_270 (Dropout)       (None, 512)               0         
                                                                 
 dense_517 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_382 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_271 (Dropout)       (None, 256)               0         
                                                                 
 dense_518 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_383 (B  (None, 128)               512       
 atchNorma

 batch_normalization_386 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_523 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
10/13 [======================>.......] - ETA: 0s - loss: 0.9405 - accuracy: 0.5063
Epoch 1: val_loss improved from inf to 0.69371, saving model to OpenAI_MLP_XGB/visualizations_mlp/CVX\best_mlp_title_L_label_layer_1.weights.h5
13/13 [==============================] - 2s 30ms/step - loss: 0.9179 - accuracy: 0.4938 - val_loss: 0.6937 - val_accuracy: 0.4733
Epoch 2/100
10/13 [======================>.......] - ETA: 0s - loss: 0.8961 - accuracy: 0.5031
Epoch 2: val_lo

17/17 [==============================] - 0s 13ms/step - loss: 0.5171 - accuracy: 0.7594 - val_loss: 1.0111 - val_accuracy: 0.2632
Epoch 10/100
15/17 [=========================>....] - ETA: 0s - loss: 0.5423 - accuracy: 0.7208
Epoch 10: val_loss did not improve from 0.73995
17/17 [==============================] - 0s 14ms/step - loss: 0.5393 - accuracy: 0.7218 - val_loss: 1.0268 - val_accuracy: 0.2632
Epoch 11/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4658 - accuracy: 0.8008
Epoch 11: val_loss did not improve from 0.73995
17/17 [==============================] - 0s 13ms/step - loss: 0.4629 - accuracy: 0.8008 - val_loss: 1.0390 - val_accuracy: 0.2632
Epoch 12/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4598 - accuracy: 0.7656
Epoch 12: val_loss did not improve from 0.73995
17/17 [==============================] - 0s 13ms/step - loss: 0.4648 - accuracy: 0.7594 - val_loss: 1.0522 - val_accuracy: 0.2632
Epoch 13/100
16/17 [===========================>

...

                                                                 
 batch_normalization_406 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_287 (Dropout)       (None, 256)               0         
                                                                 
 dense_550 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_407 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_551 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_______________________________________

[57]	validation_0-auc:0.38454
[58]	validation_0-auc:0.37672
[59]	validation_0-auc:0.37340
[60]	validation_0-auc:0.38028
[61]	validation_0-auc:0.38383
[62]	validation_0-auc:0.37933
[63]	validation_0-auc:0.37767
[64]	validation_0-auc:0.38573
[65]	validation_0-auc:0.38146
[66]	validation_0-auc:0.37648
[67]	validation_0-auc:0.37482
[68]	validation_0-auc:0.37364
[69]	validation_0-auc:0.37150
[70]	validation_0-auc:0.37553
[71]	validation_0-auc:0.37790
[72]	validation_0-auc:0.38051
[73]	validation_0-auc:0.38407
[74]	validation_0-auc:0.38620
[75]	validation_0-auc:0.38407
[76]	validation_0-auc:0.38525
[77]	validation_0-auc:0.39142
[78]	validation_0-auc:0.40043
[79]	validation_0-auc:0.39545
Test Accuracy for Layer 1: 0.6090
Test AUC for Layer 1: 0.4734

Layer 2:
Training period: 01/11/2014 - 31/10/2021
Validation period: 01/11/2021 - 31/10/2022
Testing period: 01/11/2022 - 31/10/2023
Training data: 532 samples
Validation data: 133 samples
Test data: 101 samples
Embeddings shapes - Train: (532, 1

[26]	validation_0-auc:0.42769
[27]	validation_0-auc:0.42390
[28]	validation_0-auc:0.42366
[29]	validation_0-auc:0.42627
[30]	validation_0-auc:0.43030
[31]	validation_0-auc:0.43575
[32]	validation_0-auc:0.44168
[33]	validation_0-auc:0.42722
[34]	validation_0-auc:0.42769
[35]	validation_0-auc:0.42437
[36]	validation_0-auc:0.42485
[37]	validation_0-auc:0.42461
[38]	validation_0-auc:0.43480
[39]	validation_0-auc:0.43125
[40]	validation_0-auc:0.43101
[41]	validation_0-auc:0.43528
[42]	validation_0-auc:0.42982
[43]	validation_0-auc:0.43860
[44]	validation_0-auc:0.44476
[45]	validation_0-auc:0.44618
[46]	validation_0-auc:0.45211
[47]	validation_0-auc:0.45116
[48]	validation_0-auc:0.44855
[49]	validation_0-auc:0.45330
[50]	validation_0-auc:0.45519
[51]	validation_0-auc:0.45092
[52]	validation_0-auc:0.45567
[53]	validation_0-auc:0.45282
[54]	validation_0-auc:0.44239
[55]	validation_0-auc:0.44429
[56]	validation_0-auc:0.43646
[57]	validation_0-auc:0.44026
[58]	validation_0-auc:0.44286
[59]	valid

[11]	validation_0-auc:0.44462
[12]	validation_0-auc:0.45185
[13]	validation_0-auc:0.45613
[14]	validation_0-auc:0.45452
[15]	validation_0-auc:0.46977
[16]	validation_0-auc:0.49465
[17]	validation_0-auc:0.49973
[18]	validation_0-auc:0.50963
[19]	validation_0-auc:0.49572
[20]	validation_0-auc:0.49759
[21]	validation_0-auc:0.49572
[22]	validation_0-auc:0.49117
[23]	validation_0-auc:0.50027
[24]	validation_0-auc:0.50147
[25]	validation_0-auc:0.50107
[26]	validation_0-auc:0.50696
[27]	validation_0-auc:0.51632
[28]	validation_0-auc:0.51445
[29]	validation_0-auc:0.51364
[30]	validation_0-auc:0.52622
[31]	validation_0-auc:0.52996
[32]	validation_0-auc:0.53237
[33]	validation_0-auc:0.52782
[34]	validation_0-auc:0.51097
[35]	validation_0-auc:0.50776
[36]	validation_0-auc:0.50562
[37]	validation_0-auc:0.49906
[38]	validation_0-auc:0.49050
[39]	validation_0-auc:0.49973
[40]	validation_0-auc:0.49973
[41]	validation_0-auc:0.50669
[42]	validation_0-auc:0.50428
[43]	validation_0-auc:0.50803
[44]	valid

MLP summary comparison visualization saved as: OpenAI_MLP_XGB/visualizations_summary/CVX\mlp_performance_comparison.png
XGBoost summary comparison visualization saved as: OpenAI_MLP_XGB/visualizations_summary/CVX\xgb_performance_comparison.png
MLP layer performance visualization saved as: OpenAI_MLP_XGB/visualizations_summary/CVX\mlp_layer_performance.png
XGBoost layer performance visualization saved as: OpenAI_MLP_XGB/visualizations_summary/CVX\xgboost_layer_performance.png

Analysis complete! Results saved to 'OpenAI_MLP_XGB' directory.


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP_XGB/visualizations_mlp/MPC'
        self.xgb_viz_dir = 'OpenAI_MLP_XGB/visualizations_xgb/MPC'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs(self.xgb_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP_XGB/visualizations_summary/MPC', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def get_xgb_parameters(self):
        """
        Get XGBoost parameters optimized for high-dimensional embeddings.
        Uses a single parameter set for both Title and Full text embeddings since they have the same dimension (1536).
        
        Returns:
            Dictionary of base parameters
        """
        # Setup XGBoost base parameters optimized for high-dimensional embeddings
        base_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'n_estimators': 80,
            'max_depth': 3,
            'learning_rate': 0.0001,
            'subsample': 0.7,          # Row subsampling to prevent overfitting
            'colsample_bytree': 0.5,   # Column subsampling to handle high dimensionality
            'min_child_weight': 3,     # Prevents overfitting on high-dimensional embeddings
            'reg_alpha': 1.0,          # L1 regularization
            'reg_lambda': 2.0,         # L2 regularization
            'random_state': 42,
            'use_label_encoder': False # Avoid deprecation warning
        }
        
        return base_params
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP' or 'XGBoost')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            else:  # XGBoost - simplified approach to fix API issues
                # Get XGBoost parameters
                base_params = self.get_xgb_parameters()
                
                # Create and train XGBoost model
                print(f"Creating and training XGBoost model...")
                model = xgb.XGBClassifier(**base_params)
                
                # Only use validation set for evaluation (not training set)
                eval_set = [(X_val, y_val)]
                
                # Train the model
                model.fit(
                    X_train, y_train,
                    eval_set=eval_set,
                    verbose=True
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create a simple summary for XGBoost (no detailed learning curves available)
                plt.figure(figsize=(10, 6))
                plt.text(0.5, 0.5, f'XGBoost Model Trained Successfully\nAccuracy: {accuracy_score(y_test, y_pred):.4f}\nAUC: {roc_auc_score(y_test, y_pred_proba):.4f}',
                         ha='center', va='center', size=14, fontweight='bold')
                plt.title(f'XGBoost Results ({display_text}, {label_col}, Layer {i+1})')
                plt.axis('off')
                plt.tight_layout()
                plt.savefig(f"{visualization_dir}/xgb_{display_text.replace(' ', '_')}_{label_col}_layer_{i+1}.png")
                plt.close()
                
                # Use None for training history since detailed learning curves aren't available
                training_history = None
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP', 'XGBoost']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        xgb_data = df[df['Model'] == 'XGBoost']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/MPC', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # 2. Performance comparison for XGBoost
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for XGBoost
        x = np.arange(len(xgb_data))
        
        plt.bar(x - width/2, xgb_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, xgb_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('XGBoost Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of XGBoost Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in xgb_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(xgb_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(xgb_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(xgb_data['Avg Accuracy'].max(), xgb_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/MPC', "xgb_performance_comparison.png")
        plt.savefig(save_path)
        print(f"XGBoost summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP and XGBoost models."""
        # Prepare data for visualization
        mlp_layer_data = []
        xgb_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
                else:  # XGBoost
                    xgb_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
        self._create_model_layer_visualization(xgb_layer_data, 'XGBoost')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/MPC', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP_XGB/visualizations_mlp/MPC', 'OpenAI_MLP_XGB/visualizations_xgb/MPC', 
                      'OpenAI_MLP_XGB/visualizations_summary/MPC']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_semantics_SP500_database/wall_street_news_semantics_MPC_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP_XGB' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {1: 460, 0: 378}
Class distribution for long-term prediction: {1: 503, 0: 335}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_145"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   

 dense_560 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_414 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_292 (Dropout)       (None, 512)               0         
                                                                 
 dense_561 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_415 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_293 (Dropout)       (None, 256)               0         
                                                                 
 dense_562 (Dense)           (None, 128)               32896     
          


Epoch 22: val_loss did not improve from 0.68787
17/17 [==============================] - 0s 14ms/step - loss: 0.2893 - accuracy: 0.9041 - val_loss: 0.7155 - val_accuracy: 0.5414
Epoch 22: early stopping
4/4 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_Title_S_label_layer_2.png
Test Accuracy for Layer 2: 0.5347
Test AUC for Layer 2: 0.4350

Layer 3:
Training period: 01/11/2014 - 31/10/2022
Validation period: 01/11/2022 - 31/10/2023
Testing period: 01/11/2023 - 01/11/2024
Training data: 666 samples
Validation data: 101 samples
Test data: 70 samples
Embeddings shapes - Train: (666, 1536), Val: (101, 1536), Test: (70, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_147"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_564 (Dense)           (None, 512)               786944    
             


Epoch 16: val_loss did not improve from 0.70761
13/13 [==============================] - 0s 15ms/step - loss: 0.3864 - accuracy: 0.8454 - val_loss: 0.7650 - val_accuracy: 0.3053
Epoch 16: early stopping
5/5 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_Title_L_label_layer_1.png
Test Accuracy for Layer 1: 0.2632
Test AUC for Layer 1: 0.5032

Layer 2:
Training period: 01/11/2014 - 31/10/2021
Validation period: 01/11/2021 - 31/10/2022
Testing period: 01/11/2022 - 31/10/2023
Training data: 532 samples
Validation data: 133 samples
Test data: 101 samples
Embeddings shapes - Train: (532, 1536), Val: (133, 1536), Test: (101, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (L_label)
Model: "sequential_149"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_572 (Dense)           (None, 512)               786944    
           

Epoch 15/100
20/21 [===========================>..] - ETA: 0s - loss: 0.4026 - accuracy: 0.8328
Epoch 15: val_loss did not improve from 0.65775
21/21 [==============================] - 0s 13ms/step - loss: 0.4033 - accuracy: 0.8318 - val_loss: 0.6579 - val_accuracy: 0.6139
Epoch 16/100
19/21 [==========================>...] - ETA: 0s - loss: 0.3810 - accuracy: 0.8487
Epoch 16: val_loss improved from 0.65775 to 0.65757, saving model to OpenAI_MLP_XGB/visualizations_mlp/MPC\best_mlp_title_L_label_layer_3.weights.h5
21/21 [==============================] - 0s 13ms/step - loss: 0.3875 - accuracy: 0.8423 - val_loss: 0.6576 - val_accuracy: 0.6139
Epoch 17/100
20/21 [===========================>..] - ETA: 0s - loss: 0.3880 - accuracy: 0.8250
Epoch 17: val_loss did not improve from 0.65757
21/21 [==============================] - 0s 14ms/step - loss: 0.3844 - accuracy: 0.8258 - val_loss: 0.6591 - val_accuracy: 0.6139
Epoch 18/100
21/21 [==============================] - ETA: 0s - loss: 0.3648 

...

 batch_normalization_438 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_308 (Dropout)       (None, 512)               0         
                                                                 
 dense_593 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_439 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_309 (Dropout)       (None, 256)               0         
                                                                 
 dense_594 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_440 (B  (None, 128)               512       
 atchNorma

Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
16/17 [===========================>..] - ETA: 0s - loss: 0.9258 - accuracy: 0.5020
Epoch 1: val_loss improved from inf to 0.71755, saving model to OpenAI_MLP_XGB/visualizations_mlp/MPC\best_mlp_full_text_L_label_layer_2.weights.h5
17/17 [==============================] - 2s 25ms/step - loss: 0.9217 - accuracy: 0.5056 - val_loss: 0.7175 - val_accuracy: 0.2632
Epoch 2/100
16/17 [===========================>..] - ETA: 0s - loss: 0.8382 - accuracy: 0.5488
Epoch 2: val_loss did not improve from 0.71755
17/17 [==============================] - 0s 14ms/step - loss: 0.8358 - accuracy: 0.5489 - val_loss: 0.7348 - val_accuracy: 0.2632
Epoch 3/100
16/17 [===========================>..] - ETA: 0s - loss: 0.7411 - accuracy: 0.5938
Epoch 3: val_loss did not improve from 0.71755
17/17 [===============

Epoch 29/100
21/21 [==============================] - ETA: 0s - loss: 0.2267 - accuracy: 0.9204
Epoch 29: val_loss did not improve from 0.65441
21/21 [==============================] - 0s 13ms/step - loss: 0.2267 - accuracy: 0.9204 - val_loss: 0.6759 - val_accuracy: 0.6040
Epoch 30/100
18/21 [========================>.....] - ETA: 0s - loss: 0.2325 - accuracy: 0.9219
Epoch 30: val_loss did not improve from 0.65441
21/21 [==============================] - 0s 12ms/step - loss: 0.2351 - accuracy: 0.9189 - val_loss: 0.6844 - val_accuracy: 0.6139
Epoch 31/100
21/21 [==============================] - ETA: 0s - loss: 0.2163 - accuracy: 0.9234
Epoch 31: val_loss did not improve from 0.65441
21/21 [==============================] - 0s 13ms/step - loss: 0.2163 - accuracy: 0.9234 - val_loss: 0.6845 - val_accuracy: 0.6040
Epoch 32/100
18/21 [========================>.....] - ETA: 0s - loss: 0.2211 - accuracy: 0.9236Restoring model weights from the end of the best epoch: 17.

Epoch 32: val_loss did

[6]	validation_0-auc:0.56266
[7]	validation_0-auc:0.59181
[8]	validation_0-auc:0.59553
[9]	validation_0-auc:0.61993
[10]	validation_0-auc:0.58644
[11]	validation_0-auc:0.56121
[12]	validation_0-auc:0.56307
[13]	validation_0-auc:0.55252
[14]	validation_0-auc:0.58065
[15]	validation_0-auc:0.54363
[16]	validation_0-auc:0.53929
[17]	validation_0-auc:0.52192
[18]	validation_0-auc:0.52316
[19]	validation_0-auc:0.53991
[20]	validation_0-auc:0.52771
[21]	validation_0-auc:0.52192
[22]	validation_0-auc:0.51261
[23]	validation_0-auc:0.51489
[24]	validation_0-auc:0.51406
[25]	validation_0-auc:0.50124
[26]	validation_0-auc:0.51427
[27]	validation_0-auc:0.51572
[28]	validation_0-auc:0.51799
[29]	validation_0-auc:0.52523
[30]	validation_0-auc:0.53102
[31]	validation_0-auc:0.52833
[32]	validation_0-auc:0.52978
[33]	validation_0-auc:0.52667
[34]	validation_0-auc:0.52502
[35]	validation_0-auc:0.52068
[36]	validation_0-auc:0.51861
[37]	validation_0-auc:0.52357
[38]	validation_0-auc:0.52026
[39]	validatio

Training data: 666 samples
Validation data: 101 samples
Test data: 70 samples
Embeddings shapes - Train: (666, 1536), Val: (101, 1536), Test: (70, 1536)
Creating and training XGBoost model...
[0]	validation_0-auc:0.50020
[1]	validation_0-auc:0.54058
[2]	validation_0-auc:0.55713
[3]	validation_0-auc:0.57742
[4]	validation_0-auc:0.58649
[5]	validation_0-auc:0.56738
[6]	validation_0-auc:0.53704
[7]	validation_0-auc:0.55162
[8]	validation_0-auc:0.59338
[9]	validation_0-auc:0.60126
[10]	validation_0-auc:0.58156
[11]	validation_0-auc:0.58806
[12]	validation_0-auc:0.59456
[13]	validation_0-auc:0.56974
[14]	validation_0-auc:0.54610
[15]	validation_0-auc:0.53999
[16]	validation_0-auc:0.51872
[17]	validation_0-auc:0.51458
[18]	validation_0-auc:0.50709
[19]	validation_0-auc:0.50552
[20]	validation_0-auc:0.50532
[21]	validation_0-auc:0.51024
[22]	validation_0-auc:0.51478
[23]	validation_0-auc:0.52246
[24]	validation_0-auc:0.52009
[25]	validation_0-auc:0.53073
[26]	validation_0-auc:0.52522
[27]	val

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP_XGB/visualizations_mlp/SLB'
        self.xgb_viz_dir = 'OpenAI_MLP_XGB/visualizations_xgb/SLB'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs(self.xgb_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP_XGB/visualizations_summary/SLB', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def get_xgb_parameters(self):
        """
        Get XGBoost parameters optimized for high-dimensional embeddings.
        Uses a single parameter set for both Title and Full text embeddings since they have the same dimension (1536).
        
        Returns:
            Dictionary of base parameters
        """
        # Setup XGBoost base parameters optimized for high-dimensional embeddings
        base_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'n_estimators': 150,
            'max_depth': 3,
            'learning_rate': 0.00005,
            'subsample': 0.7,          # Row subsampling to prevent overfitting
            'colsample_bytree': 0.5,   # Column subsampling to handle high dimensionality
            'min_child_weight': 3,     # Prevents overfitting on high-dimensional embeddings
            'reg_alpha': 1.0,          # L1 regularization
            'reg_lambda': 2.0,         # L2 regularization
            'random_state': 42,
            'use_label_encoder': False # Avoid deprecation warning
        }
        
        return base_params
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP' or 'XGBoost')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            else:  # XGBoost - simplified approach to fix API issues
                # Get XGBoost parameters
                base_params = self.get_xgb_parameters()
                
                # Create and train XGBoost model
                print(f"Creating and training XGBoost model...")
                model = xgb.XGBClassifier(**base_params)
                
                # Only use validation set for evaluation (not training set)
                eval_set = [(X_val, y_val)]
                
                # Train the model
                model.fit(
                    X_train, y_train,
                    eval_set=eval_set,
                    verbose=True
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create a simple summary for XGBoost (no detailed learning curves available)
                plt.figure(figsize=(10, 6))
                plt.text(0.5, 0.5, f'XGBoost Model Trained Successfully\nAccuracy: {accuracy_score(y_test, y_pred):.4f}\nAUC: {roc_auc_score(y_test, y_pred_proba):.4f}',
                         ha='center', va='center', size=14, fontweight='bold')
                plt.title(f'XGBoost Results ({display_text}, {label_col}, Layer {i+1})')
                plt.axis('off')
                plt.tight_layout()
                plt.savefig(f"{visualization_dir}/xgb_{display_text.replace(' ', '_')}_{label_col}_layer_{i+1}.png")
                plt.close()
                
                # Use None for training history since detailed learning curves aren't available
                training_history = None
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP', 'XGBoost']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        xgb_data = df[df['Model'] == 'XGBoost']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/SLB', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # 2. Performance comparison for XGBoost
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for XGBoost
        x = np.arange(len(xgb_data))
        
        plt.bar(x - width/2, xgb_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, xgb_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('XGBoost Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of XGBoost Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in xgb_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(xgb_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(xgb_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(xgb_data['Avg Accuracy'].max(), xgb_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/SLB', "xgb_performance_comparison.png")
        plt.savefig(save_path)
        print(f"XGBoost summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP and XGBoost models."""
        # Prepare data for visualization
        mlp_layer_data = []
        xgb_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
                else:  # XGBoost
                    xgb_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
        self._create_model_layer_visualization(xgb_layer_data, 'XGBoost')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/SLB', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP_XGB/visualizations_mlp/SLB', 'OpenAI_MLP_XGB/visualizations_xgb/SLB', 
                      'OpenAI_MLP_XGB/visualizations_summary/SLB']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_semantics_SP500_database/wall_street_news_semantics_SLB_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP_XGB' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {0: 433, 1: 405}
Class distribution for long-term prediction: {0: 437, 1: 401}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_157"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_608 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_450 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_316 (Dropout)       (None, 512)               0         
                                                                 
 dense_609 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_451 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_317 (Dropout)       (None, 256)               0         
          

Epoch 21/100
17/17 [==============================] - ETA: 0s - loss: 0.3009 - accuracy: 0.8910
Epoch 21: val_loss did not improve from 0.69033
17/17 [==============================] - 0s 13ms/step - loss: 0.3009 - accuracy: 0.8910 - val_loss: 0.7009 - val_accuracy: 0.4962
Epoch 22/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3270 - accuracy: 0.8750
Epoch 22: val_loss did not improve from 0.69033
17/17 [==============================] - 0s 12ms/step - loss: 0.3258 - accuracy: 0.8722 - val_loss: 0.7054 - val_accuracy: 0.4812
Epoch 23/100
13/17 [=====================>........] - ETA: 0s - loss: 0.2879 - accuracy: 0.8990
Epoch 23: val_loss did not improve from 0.69033
17/17 [==============================] - 0s 12ms/step - loss: 0.2821 - accuracy: 0.9041 - val_loss: 0.7086 - val_accuracy: 0.4962
Epoch 24/100
13/17 [=====================>........] - ETA: 0s - loss: 0.2963 - accuracy: 0.9062
Epoch 24: val_loss did not improve from 0.69033
17/17 [===========================

Epoch 15/100
17/21 [=======================>......] - ETA: 0s - loss: 0.4119 - accuracy: 0.7996
Epoch 15: val_loss did not improve from 0.69471
21/21 [==============================] - 0s 12ms/step - loss: 0.4159 - accuracy: 0.8003 - val_loss: 0.7057 - val_accuracy: 0.5149
Epoch 16/100
16/21 [=====================>........] - ETA: 0s - loss: 0.3954 - accuracy: 0.8203
Epoch 16: val_loss did not improve from 0.69471
21/21 [==============================] - 0s 12ms/step - loss: 0.4165 - accuracy: 0.8048 - val_loss: 0.7064 - val_accuracy: 0.5446
Epoch 17/100
21/21 [==============================] - ETA: 0s - loss: 0.3841 - accuracy: 0.8318
Epoch 17: val_loss did not improve from 0.69471
21/21 [==============================] - 0s 12ms/step - loss: 0.3841 - accuracy: 0.8318 - val_loss: 0.7100 - val_accuracy: 0.5149
Epoch 18/100
21/21 [==============================] - ETA: 0s - loss: 0.3562 - accuracy: 0.8468
Epoch 18: val_loss did not improve from 0.69471
21/21 [===========================

17/17 [==============================] - 0s 12ms/step - loss: 0.6321 - accuracy: 0.6692 - val_loss: 0.7211 - val_accuracy: 0.3684
Epoch 8/100
16/17 [===========================>..] - ETA: 0s - loss: 0.5481 - accuracy: 0.7305
Epoch 8: val_loss did not improve from 0.70230
17/17 [==============================] - 0s 13ms/step - loss: 0.5465 - accuracy: 0.7350 - val_loss: 0.7225 - val_accuracy: 0.3684
Epoch 9/100
13/17 [=====================>........] - ETA: 0s - loss: 0.5251 - accuracy: 0.7356
Epoch 9: val_loss did not improve from 0.70230
17/17 [==============================] - 0s 11ms/step - loss: 0.5164 - accuracy: 0.7387 - val_loss: 0.7227 - val_accuracy: 0.3684
Epoch 10/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4771 - accuracy: 0.7637
Epoch 10: val_loss did not improve from 0.70230
17/17 [==============================] - 0s 13ms/step - loss: 0.4724 - accuracy: 0.7707 - val_loss: 0.7239 - val_accuracy: 0.3609
Epoch 11/100
16/17 [===========================>..] 

...

 atchNormalization)                                              
                                                                 
 dropout_337 (Dropout)       (None, 256)               0         
                                                                 
 dense_650 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_482 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_651 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
20/21 [===========================>..] - ETA: 0s - loss: 0.9238 - accur

[63]	validation_0-auc:0.48121
[64]	validation_0-auc:0.48389
[65]	validation_0-auc:0.48121
[66]	validation_0-auc:0.47572
[67]	validation_0-auc:0.47409
[68]	validation_0-auc:0.47806
[69]	validation_0-auc:0.48063
[70]	validation_0-auc:0.48436
[71]	validation_0-auc:0.49020
[72]	validation_0-auc:0.49288
[73]	validation_0-auc:0.48296
[74]	validation_0-auc:0.48471
[75]	validation_0-auc:0.48413
[76]	validation_0-auc:0.48366
[77]	validation_0-auc:0.48786
[78]	validation_0-auc:0.48751
[79]	validation_0-auc:0.48564
[80]	validation_0-auc:0.49148
[81]	validation_0-auc:0.48739
[82]	validation_0-auc:0.48401
[83]	validation_0-auc:0.48926
[84]	validation_0-auc:0.48926
[85]	validation_0-auc:0.49218
[86]	validation_0-auc:0.48950
[87]	validation_0-auc:0.48378
[88]	validation_0-auc:0.48413
[89]	validation_0-auc:0.48646
[90]	validation_0-auc:0.48763
[91]	validation_0-auc:0.48599
[92]	validation_0-auc:0.48051
[93]	validation_0-auc:0.48191
[94]	validation_0-auc:0.48284
[95]	validation_0-auc:0.48284
[96]	valid

[105]	validation_0-auc:0.55152
[106]	validation_0-auc:0.55587
[107]	validation_0-auc:0.55538
[108]	validation_0-auc:0.55525
[109]	validation_0-auc:0.55674
[110]	validation_0-auc:0.55177
[111]	validation_0-auc:0.55550
[112]	validation_0-auc:0.55338
[113]	validation_0-auc:0.55450
[114]	validation_0-auc:0.55376
[115]	validation_0-auc:0.55326
[116]	validation_0-auc:0.55824
[117]	validation_0-auc:0.56272
[118]	validation_0-auc:0.55774
[119]	validation_0-auc:0.55923
[120]	validation_0-auc:0.56010
[121]	validation_0-auc:0.55438
[122]	validation_0-auc:0.55127
[123]	validation_0-auc:0.54978
[124]	validation_0-auc:0.55202
[125]	validation_0-auc:0.55301
[126]	validation_0-auc:0.55027
[127]	validation_0-auc:0.55090
[128]	validation_0-auc:0.55276
[129]	validation_0-auc:0.55351
[130]	validation_0-auc:0.55376
[131]	validation_0-auc:0.55002
[132]	validation_0-auc:0.55301
[133]	validation_0-auc:0.55152
[134]	validation_0-auc:0.55002
[135]	validation_0-auc:0.55152
[136]	validation_0-auc:0.54953
[137]	va

[146]	validation_0-auc:0.43779
[147]	validation_0-auc:0.43487
[148]	validation_0-auc:0.43651
[149]	validation_0-auc:0.43441
Test Accuracy for Layer 1: 0.4887
Test AUC for Layer 1: 0.6656

Layer 2:
Training period: 01/11/2014 - 31/10/2021
Validation period: 01/11/2021 - 31/10/2022
Testing period: 01/11/2022 - 31/10/2023
Training data: 532 samples
Validation data: 133 samples
Test data: 101 samples
Embeddings shapes - Train: (532, 1536), Val: (133, 1536), Test: (101, 1536)
Creating and training XGBoost model...
[0]	validation_0-auc:0.48891
[1]	validation_0-auc:0.44106
[2]	validation_0-auc:0.46980
[3]	validation_0-auc:0.48371
[4]	validation_0-auc:0.46663
[5]	validation_0-auc:0.48224
[6]	validation_0-auc:0.48247
[7]	validation_0-auc:0.46437
[8]	validation_0-auc:0.45645
[9]	validation_0-auc:0.44581
[10]	validation_0-auc:0.41324
[11]	validation_0-auc:0.40328
[12]	validation_0-auc:0.38100
[13]	validation_0-auc:0.38914
[14]	validation_0-auc:0.38111
[15]	validation_0-auc:0.37579
[16]	validation

[25]	validation_0-auc:0.64723
[26]	validation_0-auc:0.65962
[27]	validation_0-auc:0.65428
[28]	validation_0-auc:0.65938
[29]	validation_0-auc:0.65986
[30]	validation_0-auc:0.65792
[31]	validation_0-auc:0.65136
[32]	validation_0-auc:0.65792
[33]	validation_0-auc:0.65075
[34]	validation_0-auc:0.64529
[35]	validation_0-auc:0.65403
[36]	validation_0-auc:0.65403
[37]	validation_0-auc:0.64832
[38]	validation_0-auc:0.64431
[39]	validation_0-auc:0.63338
[40]	validation_0-auc:0.64504
[41]	validation_0-auc:0.64480
[42]	validation_0-auc:0.64444
[43]	validation_0-auc:0.63958
[44]	validation_0-auc:0.63739
[45]	validation_0-auc:0.64128
[46]	validation_0-auc:0.63290
[47]	validation_0-auc:0.63022
[48]	validation_0-auc:0.62949
[49]	validation_0-auc:0.63630
[50]	validation_0-auc:0.64407
[51]	validation_0-auc:0.64456
[52]	validation_0-auc:0.64359
[53]	validation_0-auc:0.64662
[54]	validation_0-auc:0.63776
[55]	validation_0-auc:0.63776
[56]	validation_0-auc:0.63520
[57]	validation_0-auc:0.63435
[58]	valid

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP_XGB/visualizations_mlp/XOM'
        self.xgb_viz_dir = 'OpenAI_MLP_XGB/visualizations_xgb/XOM'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs(self.xgb_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP_XGB/visualizations_summary/XOM', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def get_xgb_parameters(self):
        """
        Get XGBoost parameters optimized for high-dimensional embeddings.
        Uses a single parameter set for both Title and Full text embeddings since they have the same dimension (1536).
        
        Returns:
            Dictionary of base parameters
        """
        # Setup XGBoost base parameters optimized for high-dimensional embeddings
        base_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'n_estimators': 100,
            'max_depth': 3,
            'learning_rate': 0.00005,
            'subsample': 0.7,          # Row subsampling to prevent overfitting
            'colsample_bytree': 0.5,   # Column subsampling to handle high dimensionality
            'min_child_weight': 3,     # Prevents overfitting on high-dimensional embeddings
            'reg_alpha': 1.0,          # L1 regularization
            'reg_lambda': 2.0,         # L2 regularization
            'random_state': 42,
            'use_label_encoder': False # Avoid deprecation warning
        }
        
        return base_params
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP' or 'XGBoost')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            else:  # XGBoost - simplified approach to fix API issues
                # Get XGBoost parameters
                base_params = self.get_xgb_parameters()
                
                # Create and train XGBoost model
                print(f"Creating and training XGBoost model...")
                model = xgb.XGBClassifier(**base_params)
                
                # Only use validation set for evaluation (not training set)
                eval_set = [(X_val, y_val)]
                
                # Train the model
                model.fit(
                    X_train, y_train,
                    eval_set=eval_set,
                    verbose=True
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create a simple summary for XGBoost (no detailed learning curves available)
                plt.figure(figsize=(10, 6))
                plt.text(0.5, 0.5, f'XGBoost Model Trained Successfully\nAccuracy: {accuracy_score(y_test, y_pred):.4f}\nAUC: {roc_auc_score(y_test, y_pred_proba):.4f}',
                         ha='center', va='center', size=14, fontweight='bold')
                plt.title(f'XGBoost Results ({display_text}, {label_col}, Layer {i+1})')
                plt.axis('off')
                plt.tight_layout()
                plt.savefig(f"{visualization_dir}/xgb_{display_text.replace(' ', '_')}_{label_col}_layer_{i+1}.png")
                plt.close()
                
                # Use None for training history since detailed learning curves aren't available
                training_history = None
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP', 'XGBoost']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        xgb_data = df[df['Model'] == 'XGBoost']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/XOM', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # 2. Performance comparison for XGBoost
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for XGBoost
        x = np.arange(len(xgb_data))
        
        plt.bar(x - width/2, xgb_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, xgb_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('XGBoost Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of XGBoost Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in xgb_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(xgb_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(xgb_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(xgb_data['Avg Accuracy'].max(), xgb_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/XOM', "xgb_performance_comparison.png")
        plt.savefig(save_path)
        print(f"XGBoost summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP and XGBoost models."""
        # Prepare data for visualization
        mlp_layer_data = []
        xgb_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
                else:  # XGBoost
                    xgb_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
        self._create_model_layer_visualization(xgb_layer_data, 'XGBoost')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP_XGB/visualizations_summary/XOM', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP_XGB/visualizations_mlp/XOM', 'OpenAI_MLP_XGB/visualizations_xgb/XOM', 
                      'OpenAI_MLP_XGB/visualizations_summary/XOM']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_semantics_SP500_database/wall_street_news_semantics_XOM_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP_XGB' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {1: 427, 0: 411}
Class distribution for long-term prediction: {1: 428, 0: 410}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_169"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   

 dense_656 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_486 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_340 (Dropout)       (None, 512)               0         
                                                                 
 dense_657 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_487 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_341 (Dropout)       (None, 256)               0         
                                                                 
 dense_658 (Dense)           (None, 128)               32896     
          

                                                                 
 batch_normalization_490 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_343 (Dropout)       (None, 256)               0         
                                                                 
 dense_662 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_491 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_663 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_______________________________________

Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
11/13 [========================>.....] - ETA: 0s - loss: 0.8772 - accuracy: 0.5114
Epoch 1: val_loss improved from inf to 0.70047, saving model to OpenAI_MLP_XGB/visualizations_mlp/XOM\best_mlp_title_L_label_layer_1.weights.h5
13/13 [==============================] - 2s 30ms/step - loss: 0.9124 - accuracy: 0.5062 - val_loss: 0.7005 - val_accuracy: 0.3130
Epoch 2/100
 9/13 [===================>..........] - ETA: 0s - loss: 0.8189 - accuracy: 0.5347
Epoch 2: val_loss did not improve from 0.70047
13/13 [==============================] - 0s 15ms/step - loss: 0.8047 - accuracy: 0.5411 - val_loss: 0.7128 - val_accuracy: 0.3130
Epoch 3/100
11/13 [========================>.....] - ETA: 0s - loss: 0.7467 - accuracy: 0.5824
Epoch 3: val_loss did not improve from 0.70047
13/13 [===================

17/17 [==============================] - 0s 13ms/step - loss: 0.7080 - accuracy: 0.6147 - val_loss: 0.7349 - val_accuracy: 0.2707
Epoch 5/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6580 - accuracy: 0.6621
Epoch 5: val_loss did not improve from 0.71194
17/17 [==============================] - 0s 13ms/step - loss: 0.6582 - accuracy: 0.6579 - val_loss: 0.7327 - val_accuracy: 0.2707
Epoch 6/100
13/17 [=====================>........] - ETA: 0s - loss: 0.6191 - accuracy: 0.6514
Epoch 6: val_loss did not improve from 0.71194
17/17 [==============================] - 0s 11ms/step - loss: 0.6327 - accuracy: 0.6504 - val_loss: 0.7306 - val_accuracy: 0.2707
Epoch 7/100
12/17 [====================>.........] - ETA: 0s - loss: 0.5388 - accuracy: 0.7292
Epoch 7: val_loss did not improve from 0.71194
17/17 [==============================] - 0s 12ms/step - loss: 0.5646 - accuracy: 0.7180 - val_loss: 0.7220 - val_accuracy: 0.2707
Epoch 8/100
16/17 [===========================>..] - E

...

Epoch 3/100
16/17 [===========================>..] - ETA: 0s - loss: 0.7531 - accuracy: 0.5684
Epoch 3: val_loss improved from 0.67380 to 0.66758, saving model to OpenAI_MLP_XGB/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 16ms/step - loss: 0.7513 - accuracy: 0.5695 - val_loss: 0.6676 - val_accuracy: 0.7293
Epoch 4/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6928 - accuracy: 0.6172
Epoch 4: val_loss improved from 0.66758 to 0.66167, saving model to OpenAI_MLP_XGB/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 17ms/step - loss: 0.6818 - accuracy: 0.6278 - val_loss: 0.6617 - val_accuracy: 0.7293
Epoch 5/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6759 - accuracy: 0.6074
Epoch 5: val_loss improved from 0.66167 to 0.65636, saving model to OpenAI_MLP_XGB/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_2.weights.h5

Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
20/21 [===========================>..] - ETA: 0s - loss: 0.8718 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.69177, saving model to OpenAI_MLP_XGB/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_3.weights.h5
21/21 [==============================] - 2s 23ms/step - loss: 0.8831 - accuracy: 0.4925 - val_loss: 0.6918 - val_accuracy: 0.5842
Epoch 2/100
21/21 [==============================] - ETA: 0s - loss: 0.7585 - accuracy: 0.5721
Epoch 2: val_loss improved from 0.69177 to 0.69112, saving model to OpenAI_MLP_XGB/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_3.weights.h5
21/21 [==============================] - 0s 15ms/step - loss: 0.7585 - accuracy: 0.5721 - val_loss: 0.6911 - val_accuracy: 0.5842
Epoch 3/100
20/21 [===========================>..] - ETA: 0

[0]	validation_0-auc:0.51856
[1]	validation_0-auc:0.53455
[2]	validation_0-auc:0.50474
[3]	validation_0-auc:0.48130
[4]	validation_0-auc:0.46179
[5]	validation_0-auc:0.45908
[6]	validation_0-auc:0.50528
[7]	validation_0-auc:0.50298
[8]	validation_0-auc:0.50190
[9]	validation_0-auc:0.49864
[10]	validation_0-auc:0.49485
[11]	validation_0-auc:0.49228
[12]	validation_0-auc:0.51165
[13]	validation_0-auc:0.50081
[14]	validation_0-auc:0.49675
[15]	validation_0-auc:0.49146
[16]	validation_0-auc:0.50759
[17]	validation_0-auc:0.52629
[18]	validation_0-auc:0.51165
[19]	validation_0-auc:0.51396
[20]	validation_0-auc:0.50556
[21]	validation_0-auc:0.49702
[22]	validation_0-auc:0.50190
[23]	validation_0-auc:0.49444
[24]	validation_0-auc:0.49539
[25]	validation_0-auc:0.48970
[26]	validation_0-auc:0.48753
[27]	validation_0-auc:0.48604
[28]	validation_0-auc:0.47602
[29]	validation_0-auc:0.47669
[30]	validation_0-auc:0.47886
[31]	validation_0-auc:0.48076
[32]	validation_0-auc:0.48821
[33]	validation_0-au

[85]	validation_0-auc:0.46530
[86]	validation_0-auc:0.46748
[87]	validation_0-auc:0.46656
[88]	validation_0-auc:0.46748
[89]	validation_0-auc:0.46473
[90]	validation_0-auc:0.46679
[91]	validation_0-auc:0.46713
[92]	validation_0-auc:0.46759
[93]	validation_0-auc:0.46862
[94]	validation_0-auc:0.46839
[95]	validation_0-auc:0.46942
[96]	validation_0-auc:0.47000
[97]	validation_0-auc:0.46862
[98]	validation_0-auc:0.46427
[99]	validation_0-auc:0.46496
Test Accuracy for Layer 2: 0.5446
Test AUC for Layer 2: 0.3759

Layer 3:
Training period: 01/11/2014 - 31/10/2022
Validation period: 01/11/2022 - 31/10/2023
Testing period: 01/11/2023 - 01/11/2024
Training data: 666 samples
Validation data: 101 samples
Test data: 70 samples
Embeddings shapes - Train: (666, 1536), Val: (101, 1536), Test: (70, 1536)
Creating and training XGBoost model...
[0]	validation_0-auc:0.51858
[1]	validation_0-auc:0.45652
[2]	validation_0-auc:0.49348
[3]	validation_0-auc:0.49190
[4]	validation_0-auc:0.48360
[5]	validation_0

XGBoost summary comparison visualization saved as: OpenAI_MLP_XGB/visualizations_summary/XOM\xgb_performance_comparison.png
MLP layer performance visualization saved as: OpenAI_MLP_XGB/visualizations_summary/XOM\mlp_layer_performance.png
XGBoost layer performance visualization saved as: OpenAI_MLP_XGB/visualizations_summary/XOM\xgboost_layer_performance.png

Analysis complete! Results saved to 'OpenAI_MLP_XGB' directory.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/COP'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/COP', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (COP)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/COP', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/COP', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/COP', 'Merged_OpenAI_MLP/visualizations_summary/COP']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_title_full_text_SP500_database/semantic/wall_street_news_semantics_COP_completed_openai_Merged.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {0: 431, 1: 407}
Class distribution for long-term prediction: {1: 440, 0: 398}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dens

Epoch 18/100
 7/13 [===============>..............] - ETA: 0s - loss: 0.3637 - accuracy: 0.8259
Epoch 18: val_loss did not improve from 0.68880
13/13 [==============================] - 0s 11ms/step - loss: 0.3947 - accuracy: 0.8080 - val_loss: 0.7010 - val_accuracy: 0.5420
Epoch 19/100
13/13 [==============================] - ETA: 0s - loss: 0.3872 - accuracy: 0.8204
Epoch 19: val_loss did not improve from 0.68880
13/13 [==============================] - 0s 12ms/step - loss: 0.3872 - accuracy: 0.8204 - val_loss: 0.7027 - val_accuracy: 0.5420
Epoch 20/100
13/13 [==============================] - ETA: 0s - loss: 0.3320 - accuracy: 0.8703
Epoch 20: val_loss did not improve from 0.68880
13/13 [==============================] - 0s 12ms/step - loss: 0.3320 - accuracy: 0.8703 - val_loss: 0.7045 - val_accuracy: 0.5420
Epoch 21/100
 7/13 [===============>..............] - ETA: 0s - loss: 0.3610 - accuracy: 0.8527
Epoch 21: val_loss did not improve from 0.68880
13/13 [===========================


Epoch 16: val_loss did not improve from 0.69768
17/17 [==============================] - 0s 12ms/step - loss: 0.4239 - accuracy: 0.8214 - val_loss: 0.9272 - val_accuracy: 0.4662
Epoch 16: early stopping
4/4 [==============================] - 0s 2ms/step
Saved learning curves: final_mlp_merged_S_label_layer_2.png
Test Accuracy for Layer 2: 0.4455
Test AUC for Layer 2: 0.5040

Layer 3:
Training period: 01/11/2014 - 31/10/2022
Validation period: 01/11/2022 - 31/10/2023
Testing period: 01/11/2023 - 01/11/2024
Training data: 666 samples
Validation data: 101 samples
Test data: 70 samples
Embeddings shapes - Train: (666, 1536), Val: (101, 1536), Test: (70, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               786944    
  

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_9 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_10 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
          

Epoch 20/100
13/13 [==============================] - ETA: 0s - loss: 0.3191 - accuracy: 0.8878
Epoch 20: val_loss did not improve from 0.63176
13/13 [==============================] - 0s 12ms/step - loss: 0.3191 - accuracy: 0.8878 - val_loss: 0.6368 - val_accuracy: 0.6718
Epoch 21/100
13/13 [==============================] - ETA: 0s - loss: 0.3304 - accuracy: 0.8653
Epoch 21: val_loss did not improve from 0.63176
13/13 [==============================] - 0s 12ms/step - loss: 0.3304 - accuracy: 0.8653 - val_loss: 0.6374 - val_accuracy: 0.6718
Epoch 22/100
13/13 [==============================] - ETA: 0s - loss: 0.3175 - accuracy: 0.8928
Epoch 22: val_loss did not improve from 0.63176
13/13 [==============================] - 0s 12ms/step - loss: 0.3175 - accuracy: 0.8928 - val_loss: 0.6386 - val_accuracy: 0.6718
Epoch 23/100
13/13 [==============================] - ETA: 0s - loss: 0.3332 - accuracy: 0.8628
Epoch 23: val_loss did not improve from 0.63176
13/13 [===========================

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_15 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_16 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_11 (Dropout)        (None, 256)               0         
          

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/CVX'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/CVX', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (CVX)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/CVX', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/CVX', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/CVX', 'Merged_OpenAI_MLP/visualizations_summary/CVX']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_title_full_text_SP500_database/semantic/wall_street_news_semantics_CVX_completed_openai_Merged.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {1: 452, 0: 386}
Class distribution for long-term prediction: {1: 464, 0: 374}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 

 dense_28 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_21 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_29 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_22 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 dense_30 (Dense)            (None, 128)               32896     
          

19/21 [==========================>...] - ETA: 0s - loss: 0.6762 - accuracy: 0.6234
Epoch 4: val_loss did not improve from 0.69082
21/21 [==============================] - 0s 10ms/step - loss: 0.6754 - accuracy: 0.6231 - val_loss: 0.7046 - val_accuracy: 0.5347
Epoch 5/100
19/21 [==========================>...] - ETA: 0s - loss: 0.6712 - accuracy: 0.6250
Epoch 5: val_loss did not improve from 0.69082
21/21 [==============================] - 0s 11ms/step - loss: 0.6692 - accuracy: 0.6201 - val_loss: 0.7115 - val_accuracy: 0.5347
Epoch 6/100
19/21 [==========================>...] - ETA: 0s - loss: 0.5979 - accuracy: 0.6530
Epoch 6: val_loss did not improve from 0.69082
21/21 [==============================] - 0s 11ms/step - loss: 0.5979 - accuracy: 0.6562 - val_loss: 0.7168 - val_accuracy: 0.5347
Epoch 7/100
19/21 [==========================>...] - ETA: 0s - loss: 0.5922 - accuracy: 0.6924
Epoch 7: val_loss did not improve from 0.69082
21/21 [==============================] - 0s 10ms/step 

Epoch 6/100
11/13 [========================>.....] - ETA: 0s - loss: 0.6426 - accuracy: 0.6648
Epoch 6: val_loss improved from 0.66484 to 0.66120, saving model to Merged_OpenAI_MLP/visualizations_mlp/CVX\best_mlp_merged_L_label_layer_1.weights.h5
13/13 [==============================] - 0s 16ms/step - loss: 0.6344 - accuracy: 0.6633 - val_loss: 0.6612 - val_accuracy: 0.6794
Epoch 7/100
12/13 [==========================>...] - ETA: 0s - loss: 0.5981 - accuracy: 0.6693
Epoch 7: val_loss improved from 0.66120 to 0.65778, saving model to Merged_OpenAI_MLP/visualizations_mlp/CVX\best_mlp_merged_L_label_layer_1.weights.h5
13/13 [==============================] - 0s 16ms/step - loss: 0.6031 - accuracy: 0.6658 - val_loss: 0.6578 - val_accuracy: 0.6794
Epoch 8/100
12/13 [==========================>...] - ETA: 0s - loss: 0.5902 - accuracy: 0.6979
Epoch 8: val_loss improved from 0.65778 to 0.65470, saving model to Merged_OpenAI_MLP/visualizations_mlp/CVX\best_mlp_merged_L_label_layer_1.weights.h5

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_30 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_41 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_31 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_21 (Dropout)        (None, 256)               0         
          

 batch_normalization_35 (Ba  (None, 128)               512       
 tchNormalization)                                               
                                                                 
 dense_47 (Dense)            (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
16/21 [=====================>........] - ETA: 0s - loss: 0.9425 - accuracy: 0.5039
Epoch 1: val_loss improved from inf to 0.70287, saving model to Merged_OpenAI_MLP/visualizations_mlp/CVX\best_mlp_merged_L_label_layer_3.weights.h5
21/21 [==============================] - 2s 21ms/step - loss: 0.9135 - accuracy: 0.5090 - val_loss: 0.7029 - val_accuracy: 0.4158
Epoch 2/100
17/21 [=======================>......] - ETA: 0s - loss: 0.8475 - accuracy: 0.5202
Epoch 2: va

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/MPC'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/MPC', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (MPC)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/MPC', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/MPC', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/MPC', 'Merged_OpenAI_MLP/visualizations_summary/MPC']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_title_full_text_SP500_database/semantic/wall_street_news_semantics_MPC_completed_openai_Merged.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {1: 460, 0: 378}
Class distribution for long-term prediction: {1: 503, 0: 335}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48

17/17 [==============================] - ETA: 0s - loss: 0.8002 - accuracy: 0.5320
Epoch 2: val_loss did not improve from 0.69361
17/17 [==============================] - 0s 12ms/step - loss: 0.8002 - accuracy: 0.5320 - val_loss: 0.6953 - val_accuracy: 0.4586
Epoch 3/100
12/17 [====================>.........] - ETA: 0s - loss: 0.7642 - accuracy: 0.5703
Epoch 3: val_loss did not improve from 0.69361
17/17 [==============================] - 0s 12ms/step - loss: 0.7766 - accuracy: 0.5602 - val_loss: 0.6967 - val_accuracy: 0.4511
Epoch 4/100
13/17 [=====================>........] - ETA: 0s - loss: 0.7096 - accuracy: 0.6106
Epoch 4: val_loss did not improve from 0.69361
17/17 [==============================] - 0s 11ms/step - loss: 0.7302 - accuracy: 0.6034 - val_loss: 0.6969 - val_accuracy: 0.4511
Epoch 5/100
12/17 [====================>.........] - ETA: 0s - loss: 0.7138 - accuracy: 0.6354
Epoch 5: val_loss did not improve from 0.69361
17/17 [==============================] - 0s 12ms/step 

21/21 [==============================] - 0s 10ms/step - loss: 0.2679 - accuracy: 0.8934 - val_loss: 0.7114 - val_accuracy: 0.5545
Epoch 31/100
19/21 [==========================>...] - ETA: 0s - loss: 0.2552 - accuracy: 0.9211
Epoch 31: val_loss did not improve from 0.67798
21/21 [==============================] - 0s 11ms/step - loss: 0.2560 - accuracy: 0.9174 - val_loss: 0.7144 - val_accuracy: 0.5446
Epoch 32/100
18/21 [========================>.....] - ETA: 0s - loss: 0.2491 - accuracy: 0.9045
Epoch 32: val_loss did not improve from 0.67798
21/21 [==============================] - 0s 11ms/step - loss: 0.2487 - accuracy: 0.9099 - val_loss: 0.7234 - val_accuracy: 0.5842
Epoch 33/100
19/21 [==========================>...] - ETA: 0s - loss: 0.2471 - accuracy: 0.9128
Epoch 33: val_loss did not improve from 0.67798
21/21 [==============================] - 0s 11ms/step - loss: 0.2485 - accuracy: 0.9099 - val_loss: 0.7335 - val_accuracy: 0.5446
Epoch 34/100
19/21 [==========================>.

Epoch 8/100
12/13 [==========================>...] - ETA: 0s - loss: 0.5429 - accuracy: 0.7161
Epoch 8: val_loss did not improve from 0.65586
13/13 [==============================] - 0s 13ms/step - loss: 0.5389 - accuracy: 0.7182 - val_loss: 0.6561 - val_accuracy: 0.6947
Epoch 9/100
11/13 [========================>.....] - ETA: 0s - loss: 0.5433 - accuracy: 0.7159
Epoch 9: val_loss did not improve from 0.65586
13/13 [==============================] - 0s 13ms/step - loss: 0.5524 - accuracy: 0.7082 - val_loss: 0.6566 - val_accuracy: 0.6947
Epoch 10/100
13/13 [==============================] - ETA: 0s - loss: 0.5108 - accuracy: 0.7481
Epoch 10: val_loss did not improve from 0.65586
13/13 [==============================] - 0s 13ms/step - loss: 0.5108 - accuracy: 0.7481 - val_loss: 0.6580 - val_accuracy: 0.6947
Epoch 11/100
13/13 [==============================] - ETA: 0s - loss: 0.4978 - accuracy: 0.7830
Epoch 11: val_loss did not improve from 0.65586
13/13 [==============================]

Epoch 6/100
17/17 [==============================] - ETA: 0s - loss: 0.6173 - accuracy: 0.6372
Epoch 6: val_loss improved from 0.63305 to 0.62730, saving model to Merged_OpenAI_MLP/visualizations_mlp/MPC\best_mlp_merged_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 14ms/step - loss: 0.6173 - accuracy: 0.6372 - val_loss: 0.6273 - val_accuracy: 0.7368
Epoch 7/100
12/17 [====================>.........] - ETA: 0s - loss: 0.5855 - accuracy: 0.6849
Epoch 7: val_loss improved from 0.62730 to 0.62366, saving model to Merged_OpenAI_MLP/visualizations_mlp/MPC\best_mlp_merged_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 14ms/step - loss: 0.5790 - accuracy: 0.6823 - val_loss: 0.6237 - val_accuracy: 0.7368
Epoch 8/100
12/17 [====================>.........] - ETA: 0s - loss: 0.5800 - accuracy: 0.6901
Epoch 8: val_loss improved from 0.62366 to 0.62210, saving model to Merged_OpenAI_MLP/visualizations_mlp/MPC\best_mlp_merged_L_label_layer_2.weights.h5

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_68 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_51 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_34 (Dropout)        (None, 512)               0         
                                                                 
 dense_69 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_52 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_35 (Dropout)        (None, 256)               0         
          

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/SLB'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/SLB', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (SLB)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/SLB', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/SLB', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/SLB', 'Merged_OpenAI_MLP/visualizations_summary/SLB']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_title_full_text_SP500_database/semantic/wall_street_news_semantics_SLB_completed_openai_Merged.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {0: 433, 1: 405}
Class distribution for long-term prediction: {0: 437, 1: 401}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_72

 dense_76 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_57 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_38 (Dropout)        (None, 512)               0         
                                                                 
 dense_77 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_58 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_39 (Dropout)        (None, 256)               0         
                                                                 
 dense_78 (Dense)            (None, 128)               32896     
          

Epoch 18/100
18/21 [========================>.....] - ETA: 0s - loss: 0.3953 - accuracy: 0.8385
Epoch 18: val_loss did not improve from 0.69058
21/21 [==============================] - 0s 11ms/step - loss: 0.4017 - accuracy: 0.8348 - val_loss: 0.7249 - val_accuracy: 0.4554
Epoch 19/100
18/21 [========================>.....] - ETA: 0s - loss: 0.3838 - accuracy: 0.8420
Epoch 19: val_loss did not improve from 0.69058
21/21 [==============================] - 0s 12ms/step - loss: 0.3836 - accuracy: 0.8393 - val_loss: 0.7312 - val_accuracy: 0.4158
Epoch 20/100
19/21 [==========================>...] - ETA: 0s - loss: 0.3496 - accuracy: 0.8553
Epoch 20: val_loss did not improve from 0.69058
21/21 [==============================] - 0s 11ms/step - loss: 0.3456 - accuracy: 0.8619 - val_loss: 0.7415 - val_accuracy: 0.4059
Epoch 21/100
19/21 [==========================>...] - ETA: 0s - loss: 0.3643 - accuracy: 0.8553Restoring model weights from the end of the best epoch: 6.

Epoch 21: val_loss did 

                                                                 
 batch_normalization_69 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_46 (Dropout)        (None, 512)               0         
                                                                 
 dense_93 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_70 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_47 (Dropout)        (None, 256)               0         
                                                                 
 dense_94 (Dense)            (None, 128)               32896     
                                                                 
 batch_nor

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/XOM'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/XOM', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2014-2020), validation (2020-2021), test (2021-2022)
        layer1 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2020-10-31'),
            'val_start': pd.Timestamp('2020-11-01'),
            'val_end': pd.Timestamp('2021-10-31'),
            'test_start': pd.Timestamp('2021-11-01'),
            'test_end': pd.Timestamp('2022-10-31')
        }
        
        # Second layer: train (2014-2021), validation (2021-2022), test (2022-2023)
        layer2 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2021-10-31'),
            'val_start': pd.Timestamp('2021-11-01'),
            'val_end': pd.Timestamp('2022-10-31'),
            'test_start': pd.Timestamp('2022-11-01'),
            'test_end': pd.Timestamp('2023-10-31')
        }
        
        # Third layer: train (2014-2022), validation (2022-2023), test (2023-2024)
        layer3 = {
            'train_start': pd.Timestamp('2014-11-01'),
            'train_end': pd.Timestamp('2022-10-31'),
            'val_start': pd.Timestamp('2022-11-01'),
            'val_end': pd.Timestamp('2023-10-31'),
            'test_start': pd.Timestamp('2023-11-01'),
            'test_end': pd.Timestamp('2024-11-01')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (XOM)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/XOM', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/XOM', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/XOM', 'Merged_OpenAI_MLP/visualizations_summary/XOM']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/wall_street_news_title_full_text_SP500_database/semantic/wall_street_news_semantics_XOM_completed_openai_Merged.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 838 climate change news articles spanning from 07/11/2014 to 24/09/2024
Class distribution for short-term prediction: {1: 427, 0: 411}
Class distribution for long-term prediction: {1: 428, 0: 410}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/11/2014 - 31/10/2020
Validation period: 01/11/2020 - 31/10/2021
Testing period: 01/11/2021 - 31/10/2022
Training data: 401 samples
Validation data: 131 samples
Test data: 133 samples
Embeddings shapes - Train: (401, 1536), Val: (131, 1536), Test: (133, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_96

 dense_100 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_75 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_50 (Dropout)        (None, 512)               0         
                                                                 
 dense_101 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_76 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_51 (Dropout)        (None, 256)               0         
                                                                 
 dense_102 (Dense)           (None, 128)               32896     
          

 dense_107 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
16/21 [=====================>........] - ETA: 0s - loss: 0.8824 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.70349, saving model to Merged_OpenAI_MLP/visualizations_mlp/XOM\best_mlp_merged_S_label_layer_3.weights.h5
21/21 [==============================] - 2s 22ms/step - loss: 0.8736 - accuracy: 0.5075 - val_loss: 0.7035 - val_accuracy: 0.4554
Epoch 2/100
18/21 [========================>.....] - ETA: 0s - loss: 0.8463 - accuracy: 0.5347
Epoch 2: val_loss did not improve from 0.70349
21/21 [==============================] - 0s 11ms/step - loss: 0.8487 - accuracy: 0.5285 - val_loss: 0.7183 - val_accuracy: 0.4554
Epoch 3/100
18/21 [=============

13/13 [==============================] - 0s 14ms/step - loss: 0.8564 - accuracy: 0.5287 - val_loss: 0.7906 - val_accuracy: 0.3130
Epoch 3/100
12/13 [==========================>...] - ETA: 0s - loss: 0.7567 - accuracy: 0.5703
Epoch 3: val_loss did not improve from 0.73475
13/13 [==============================] - 0s 13ms/step - loss: 0.7519 - accuracy: 0.5761 - val_loss: 0.8484 - val_accuracy: 0.3130
Epoch 4/100
12/13 [==========================>...] - ETA: 0s - loss: 0.7162 - accuracy: 0.5859
Epoch 4: val_loss did not improve from 0.73475
13/13 [==============================] - 0s 13ms/step - loss: 0.7196 - accuracy: 0.5810 - val_loss: 0.9114 - val_accuracy: 0.3130
Epoch 5/100
12/13 [==========================>...] - ETA: 0s - loss: 0.6228 - accuracy: 0.6562
Epoch 5: val_loss did not improve from 0.73475
13/13 [==============================] - 0s 13ms/step - loss: 0.6280 - accuracy: 0.6559 - val_loss: 0.9722 - val_accuracy: 0.3130
Epoch 6/100
13/13 [==============================] - E

17/17 [==============================] - 0s 15ms/step - loss: 0.6457 - accuracy: 0.6429 - val_loss: 0.6545 - val_accuracy: 0.7293
Epoch 6/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6516 - accuracy: 0.6152
Epoch 6: val_loss improved from 0.65446 to 0.64766, saving model to Merged_OpenAI_MLP/visualizations_mlp/XOM\best_mlp_merged_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 15ms/step - loss: 0.6443 - accuracy: 0.6184 - val_loss: 0.6477 - val_accuracy: 0.7293
Epoch 7/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6066 - accuracy: 0.6855
Epoch 7: val_loss improved from 0.64766 to 0.64184, saving model to Merged_OpenAI_MLP/visualizations_mlp/XOM\best_mlp_merged_L_label_layer_2.weights.h5
17/17 [==============================] - 0s 16ms/step - loss: 0.6055 - accuracy: 0.6880 - val_loss: 0.6418 - val_accuracy: 0.7293
Epoch 8/100
17/17 [==============================] - ETA: 0s - loss: 0.5833 - accuracy: 0.6823
Epoch 8: val_loss imp

17/17 [==============================] - 0s 12ms/step - loss: 0.2504 - accuracy: 0.9248 - val_loss: 0.6716 - val_accuracy: 0.6015
Epoch 31/100
13/17 [=====================>........] - ETA: 0s - loss: 0.2343 - accuracy: 0.9303
Epoch 31: val_loss did not improve from 0.61312
17/17 [==============================] - 0s 11ms/step - loss: 0.2373 - accuracy: 0.9267 - val_loss: 0.6839 - val_accuracy: 0.5714
Epoch 32/100
12/17 [====================>.........] - ETA: 0s - loss: 0.2291 - accuracy: 0.9271
Epoch 32: val_loss did not improve from 0.61312
17/17 [==============================] - 0s 11ms/step - loss: 0.2260 - accuracy: 0.9323 - val_loss: 0.6945 - val_accuracy: 0.5714
Epoch 33/100
12/17 [====================>.........] - ETA: 0s - loss: 0.2239 - accuracy: 0.9219Restoring model weights from the end of the best epoch: 18.

Epoch 33: val_loss did not improve from 0.61312
17/17 [==============================] - 0s 11ms/step - loss: 0.2410 - accuracy: 0.9023 - val_loss: 0.7056 - val_accur

MLP layer performance visualization saved as: Merged_OpenAI_MLP/visualizations_summary/XOM\mlp_merged_layer_performance.png

Analysis complete! Results saved to 'Merged_OpenAI_MLP' directory.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP/visualizations_mlp/XOM'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP/visualizations_summary/XOM', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS （XOM)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/XOM', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        mlp_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/XOM', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP/visualizations_mlp/XOM',
                      'OpenAI_MLP/visualizations_summary/XOM']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_XOM_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {0: 466, 1: 463}
Class distribution for long-term prediction: {1: 507, 0: 422}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dens

Epoch 19/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3557 - accuracy: 0.8822
Epoch 19: val_loss did not improve from 0.69135
17/17 [==============================] - 0s 12ms/step - loss: 0.3567 - accuracy: 0.8793 - val_loss: 0.6934 - val_accuracy: 0.5153
Epoch 20/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3444 - accuracy: 0.8726
Epoch 20: val_loss did not improve from 0.69135
17/17 [==============================] - 0s 12ms/step - loss: 0.3392 - accuracy: 0.8755 - val_loss: 0.6931 - val_accuracy: 0.5276
Epoch 21/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3272 - accuracy: 0.8870
Epoch 21: val_loss did not improve from 0.69135
17/17 [==============================] - 0s 11ms/step - loss: 0.3176 - accuracy: 0.8870 - val_loss: 0.6926 - val_accuracy: 0.5460
Epoch 22/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3080 - accuracy: 0.8894
Epoch 22: val_loss improved from 0.69135 to 0.69077, saving model to OpenAI_MLP/vi

21/22 [===========================>..] - ETA: 0s - loss: 0.8577 - accuracy: 0.5164
Epoch 1: val_loss improved from inf to 0.67951, saving model to OpenAI_MLP/visualizations_mlp/XOM\best_mlp_title_S_label_layer_2.weights.h5
22/22 [==============================] - 2s 20ms/step - loss: 0.8621 - accuracy: 0.5124 - val_loss: 0.6795 - val_accuracy: 0.6212
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.8016 - accuracy: 0.5504
Epoch 2: val_loss improved from 0.67951 to 0.67366, saving model to OpenAI_MLP/visualizations_mlp/XOM\best_mlp_title_S_label_layer_2.weights.h5
22/22 [==============================] - 0s 13ms/step - loss: 0.8016 - accuracy: 0.5504 - val_loss: 0.6737 - val_accuracy: 0.6212
Epoch 3/100
18/22 [=======================>......] - ETA: 0s - loss: 0.6945 - accuracy: 0.6007
Epoch 3: val_loss improved from 0.67366 to 0.67100, saving model to OpenAI_MLP/visualizations_mlp/XOM\best_mlp_title_S_label_layer_2.weights.h5
22/22 [==============================] 

24/24 [==============================] - 2s 20ms/step - loss: 0.8777 - accuracy: 0.5060 - val_loss: 0.7093 - val_accuracy: 0.4352
Epoch 2/100
24/24 [==============================] - ETA: 0s - loss: 0.8336 - accuracy: 0.5166
Epoch 2: val_loss did not improve from 0.70933
24/24 [==============================] - 0s 11ms/step - loss: 0.8336 - accuracy: 0.5166 - val_loss: 0.7306 - val_accuracy: 0.4352
Epoch 3/100
19/24 [======================>.......] - ETA: 0s - loss: 0.7635 - accuracy: 0.5609
Epoch 3: val_loss did not improve from 0.70933
24/24 [==============================] - 0s 11ms/step - loss: 0.7692 - accuracy: 0.5539 - val_loss: 0.7558 - val_accuracy: 0.4352
Epoch 4/100
19/24 [======================>.......] - ETA: 0s - loss: 0.6678 - accuracy: 0.6382
Epoch 4: val_loss did not improve from 0.70933
24/24 [==============================] - 0s 10ms/step - loss: 0.6764 - accuracy: 0.6338 - val_loss: 0.7823 - val_accuracy: 0.4352
Epoch 5/100
19/24 [======================>.......] - E

Epoch 5/100
12/17 [====================>.........] - ETA: 0s - loss: 0.7181 - accuracy: 0.6042
Epoch 5: val_loss did not improve from 0.69669
17/17 [==============================] - 0s 12ms/step - loss: 0.7014 - accuracy: 0.6169 - val_loss: 0.7316 - val_accuracy: 0.5031
Epoch 6/100
13/17 [=====================>........] - ETA: 0s - loss: 0.5988 - accuracy: 0.6683
Epoch 6: val_loss did not improve from 0.69669
17/17 [==============================] - 0s 11ms/step - loss: 0.5995 - accuracy: 0.6762 - val_loss: 0.7389 - val_accuracy: 0.5031
Epoch 7/100
13/17 [=====================>........] - ETA: 0s - loss: 0.5914 - accuracy: 0.6923
Epoch 7: val_loss did not improve from 0.69669
17/17 [==============================] - 0s 11ms/step - loss: 0.5777 - accuracy: 0.7088 - val_loss: 0.7458 - val_accuracy: 0.5031
Epoch 8/100
13/17 [=====================>........] - ETA: 0s - loss: 0.5737 - accuracy: 0.7067
Epoch 8: val_loss did not improve from 0.69669
17/17 [==============================] - 0

...

24/24 [==============================] - 0s 11ms/step - loss: 0.3250 - accuracy: 0.8828 - val_loss: 0.7186 - val_accuracy: 0.5370
Epoch 24/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3598 - accuracy: 0.8372Restoring model weights from the end of the best epoch: 9.

Epoch 24: val_loss did not improve from 0.67902
24/24 [==============================] - 0s 10ms/step - loss: 0.3597 - accuracy: 0.8375 - val_loss: 0.7216 - val_accuracy: 0.5278
Epoch 24: early stopping
3/3 [==============================] - 0s 2ms/step
Saved learning curves: final_mlp_Full_text_S_label_layer_3.png
Test Accuracy for Layer 3: 0.5072
Test AUC for Layer 3: 0.4613

Average Test Accuracy across all layers: 0.4404
Average Test AUC across all layers: 0.5220

Training MLP model for Full text and L_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test da

Epoch 17/100
12/17 [====================>.........] - ETA: 0s - loss: 0.4018 - accuracy: 0.8099
Epoch 17: val_loss did not improve from 0.69144
17/17 [==============================] - 0s 12ms/step - loss: 0.3969 - accuracy: 0.8180 - val_loss: 0.6924 - val_accuracy: 0.5460
Epoch 18/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3686 - accuracy: 0.8413
Epoch 18: val_loss did not improve from 0.69144
17/17 [==============================] - 0s 11ms/step - loss: 0.3746 - accuracy: 0.8391 - val_loss: 0.6927 - val_accuracy: 0.5521
Epoch 19/100
14/17 [=======================>......] - ETA: 0s - loss: 0.3572 - accuracy: 0.8549
Epoch 19: val_loss did not improve from 0.69144
17/17 [==============================] - 0s 11ms/step - loss: 0.3613 - accuracy: 0.8467 - val_loss: 0.6940 - val_accuracy: 0.5583
Epoch 20/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3533 - accuracy: 0.8486
Epoch 20: val_loss did not improve from 0.69144
17/17 [===========================

21/22 [===========================>..] - ETA: 0s - loss: 0.5908 - accuracy: 0.6830
Epoch 8: val_loss improved from 0.52873 to 0.52668, saving model to OpenAI_MLP/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 14ms/step - loss: 0.5894 - accuracy: 0.6818 - val_loss: 0.5267 - val_accuracy: 0.8636
Epoch 9/100
17/22 [======================>.......] - ETA: 0s - loss: 0.5443 - accuracy: 0.7077
Epoch 9: val_loss improved from 0.52668 to 0.52137, saving model to OpenAI_MLP/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 14ms/step - loss: 0.5480 - accuracy: 0.7066 - val_loss: 0.5214 - val_accuracy: 0.8636
Epoch 10/100
20/22 [==========================>...] - ETA: 0s - loss: 0.5291 - accuracy: 0.7500
Epoch 10: val_loss improved from 0.52137 to 0.52128, saving model to OpenAI_MLP/visualizations_mlp/XOM\best_mlp_full_text_L_label_layer_2.weights.h5
22/22 [==============

 dense_45 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_34 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_23 (Dropout)        (None, 256)               0         
                                                                 
 dense_46 (Dense)            (None, 128)               32896     
                                                                 
 batch_normalization_35 (Ba  (None, 128)               512       
 tchNormalization)                                               
                                                                 
 dense_47 (Dense)            (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-traina

MLP layer performance visualization saved as: OpenAI_MLP/visualizations_summary/XOM\mlp_layer_performance.png

Analysis complete! Results saved to 'OpenAI_MLP' directory.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/XOM'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/XOM', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (XOM)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/XOM', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/XOM', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/XOM', 'Merged_OpenAI_MLP/visualizations_summary/XOM']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_XOM_completed_openai.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {0: 466, 1: 463}
Class distribution for long-term prediction: {1: 507, 0: 422}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (

 dense_52 (Dense)            (None, 512)               786944    
                                                                 
 batch_normalization_39 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_26 (Dropout)        (None, 512)               0         
                                                                 
 dense_53 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_40 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_27 (Dropout)        (None, 256)               0         
                                                                 
 dense_54 (Dense)            (None, 128)               32896     
          

 dense_59 (Dense)            (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
22/24 [==========================>...] - ETA: 0s - loss: 0.9525 - accuracy: 0.4531
Epoch 1: val_loss improved from inf to 0.69657, saving model to Merged_OpenAI_MLP/visualizations_mlp/XOM\best_mlp_merged_S_label_layer_3.weights.h5
24/24 [==============================] - 2s 20ms/step - loss: 0.9525 - accuracy: 0.4607 - val_loss: 0.6966 - val_accuracy: 0.4352
Epoch 2/100
23/24 [===========================>..] - ETA: 0s - loss: 0.8446 - accuracy: 0.5462
Epoch 2: val_loss did not improve from 0.69657
24/24 [==============================] - 0s 11ms/step - loss: 0.8484 - accuracy: 0.5433 - val_loss: 0.7011 - val_accuracy: 0.4352
Epoch 3/100
23/24 [=============

17/17 [==============================] - 0s 12ms/step - loss: 0.7766 - accuracy: 0.5460 - val_loss: 0.6928 - val_accuracy: 0.5031
Epoch 3/100
13/17 [=====================>........] - ETA: 0s - loss: 0.7562 - accuracy: 0.5673
Epoch 3: val_loss did not improve from 0.69267
17/17 [==============================] - 0s 12ms/step - loss: 0.7424 - accuracy: 0.5690 - val_loss: 0.6932 - val_accuracy: 0.5031
Epoch 4/100
13/17 [=====================>........] - ETA: 0s - loss: 0.6721 - accuracy: 0.6394
Epoch 4: val_loss did not improve from 0.69267
17/17 [==============================] - 0s 11ms/step - loss: 0.6779 - accuracy: 0.6360 - val_loss: 0.6938 - val_accuracy: 0.5031
Epoch 5/100
14/17 [=======================>......] - ETA: 0s - loss: 0.6425 - accuracy: 0.6562
Epoch 5: val_loss did not improve from 0.69267
17/17 [==============================] - 0s 11ms/step - loss: 0.6501 - accuracy: 0.6475 - val_loss: 0.6951 - val_accuracy: 0.5031
Epoch 6/100
13/17 [=====================>........] - E

18/22 [=======================>......] - ETA: 0s - loss: 0.6285 - accuracy: 0.6510
Epoch 6: val_loss did not improve from 0.70186
22/22 [==============================] - 0s 11ms/step - loss: 0.6347 - accuracy: 0.6540 - val_loss: 0.7032 - val_accuracy: 0.2879
Epoch 7/100
19/22 [========================>.....] - ETA: 0s - loss: 0.6072 - accuracy: 0.6645
Epoch 7: val_loss improved from 0.70186 to 0.70101, saving model to Merged_OpenAI_MLP/visualizations_mlp/XOM\best_mlp_merged_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 12ms/step - loss: 0.6017 - accuracy: 0.6642 - val_loss: 0.7010 - val_accuracy: 0.3333
Epoch 8/100
18/22 [=======================>......] - ETA: 0s - loss: 0.5438 - accuracy: 0.7361
Epoch 8: val_loss improved from 0.70101 to 0.69660, saving model to Merged_OpenAI_MLP/visualizations_mlp/XOM\best_mlp_merged_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 12ms/step - loss: 0.5615 - accuracy: 0.7212 - val_loss: 0.6966 - val_acc

22/22 [==============================] - 0s 11ms/step - loss: 0.2525 - accuracy: 0.9051 - val_loss: 0.7570 - val_accuracy: 0.5606
Epoch 33/100
20/22 [==========================>...] - ETA: 0s - loss: 0.2271 - accuracy: 0.9266
Epoch 33: val_loss did not improve from 0.65792
22/22 [==============================] - 0s 10ms/step - loss: 0.2233 - accuracy: 0.9285 - val_loss: 0.7635 - val_accuracy: 0.5758
Epoch 34/100
19/22 [========================>.....] - ETA: 0s - loss: 0.2142 - accuracy: 0.9457
Epoch 34: val_loss did not improve from 0.65792
22/22 [==============================] - 0s 11ms/step - loss: 0.2197 - accuracy: 0.9387 - val_loss: 0.8093 - val_accuracy: 0.5758
Epoch 35/100
19/22 [========================>.....] - ETA: 0s - loss: 0.2193 - accuracy: 0.9408Restoring model weights from the end of the best epoch: 20.

Epoch 35: val_loss did not improve from 0.65792
22/22 [==============================] - 0s 11ms/step - loss: 0.2151 - accuracy: 0.9416 - val_loss: 0.7850 - val_accur

Epoch 17/100
24/24 [==============================] - ETA: 0s - loss: 0.3739 - accuracy: 0.8336
Epoch 17: val_loss did not improve from 0.63881
24/24 [==============================] - 0s 11ms/step - loss: 0.3739 - accuracy: 0.8336 - val_loss: 0.6629 - val_accuracy: 0.6667
Epoch 18/100
21/24 [=========================>....] - ETA: 0s - loss: 0.3738 - accuracy: 0.8438
Epoch 18: val_loss did not improve from 0.63881
24/24 [==============================] - 0s 12ms/step - loss: 0.3678 - accuracy: 0.8495 - val_loss: 0.6724 - val_accuracy: 0.6574
Epoch 19/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3701 - accuracy: 0.8454Restoring model weights from the end of the best epoch: 4.

Epoch 19: val_loss did not improve from 0.63881
24/24 [==============================] - 0s 11ms/step - loss: 0.3715 - accuracy: 0.8362 - val_loss: 0.6705 - val_accuracy: 0.6481
Epoch 19: early stopping
3/3 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_merged_L_l

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP/visualizations_mlp/SLB'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP/visualizations_summary/SLB', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS （SLB)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/SLB', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        mlp_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/SLB', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP/visualizations_mlp/SLB',
                      'OpenAI_MLP/visualizations_summary/SLB']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_SLB_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {1: 472, 0: 457}
Class distribution for long-term prediction: {1: 494, 0: 435}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 d

Epoch 20/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3572 - accuracy: 0.8702
Epoch 20: val_loss did not improve from 0.69372
17/17 [==============================] - 0s 11ms/step - loss: 0.3602 - accuracy: 0.8716 - val_loss: 0.6951 - val_accuracy: 0.4908
Epoch 21/100
12/17 [====================>.........] - ETA: 0s - loss: 0.3257 - accuracy: 0.9036
Epoch 21: val_loss did not improve from 0.69372
17/17 [==============================] - 0s 12ms/step - loss: 0.3333 - accuracy: 0.8851 - val_loss: 0.6958 - val_accuracy: 0.4969
Epoch 22/100
17/17 [==============================] - ETA: 0s - loss: 0.3281 - accuracy: 0.8697
Epoch 22: val_loss did not improve from 0.69372
17/17 [==============================] - 0s 13ms/step - loss: 0.3281 - accuracy: 0.8697 - val_loss: 0.6973 - val_accuracy: 0.5031
Epoch 23/100
13/17 [=====================>........] - ETA: 0s - loss: 0.2978 - accuracy: 0.8966
Epoch 23: val_loss did not improve from 0.69372
17/17 [===========================

22/22 [==============================] - 0s 12ms/step - loss: 0.4320 - accuracy: 0.7942 - val_loss: 0.9004 - val_accuracy: 0.5152
Epoch 14/100
19/22 [========================>.....] - ETA: 0s - loss: 0.4165 - accuracy: 0.8109
Epoch 14: val_loss did not improve from 0.69961
22/22 [==============================] - 0s 10ms/step - loss: 0.4220 - accuracy: 0.8058 - val_loss: 0.9082 - val_accuracy: 0.5152
Epoch 15/100
19/22 [========================>.....] - ETA: 0s - loss: 0.3907 - accuracy: 0.8405
Epoch 15: val_loss did not improve from 0.69961
22/22 [==============================] - 0s 10ms/step - loss: 0.3916 - accuracy: 0.8365 - val_loss: 0.9122 - val_accuracy: 0.5152
Epoch 16/100
19/22 [========================>.....] - ETA: 0s - loss: 0.4007 - accuracy: 0.8174Restoring model weights from the end of the best epoch: 1.

Epoch 16: val_loss did not improve from 0.69961
22/22 [==============================] - 0s 11ms/step - loss: 0.4007 - accuracy: 0.8146 - val_loss: 0.9072 - val_accura

24/24 [==============================] - 0s 11ms/step - loss: 0.3781 - accuracy: 0.8628 - val_loss: 0.7256 - val_accuracy: 0.5278
Epoch 17: early stopping
3/3 [==============================] - 0s 2ms/step
Saved learning curves: final_mlp_Title_S_label_layer_3.png
Test Accuracy for Layer 3: 0.5652
Test AUC for Layer 3: 0.4726

Average Test Accuracy across all layers: 0.5262
Average Test AUC across all layers: 0.4710

Training MLP model for Title and L_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (L_label)
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param 

Epoch 19/100
16/17 [===========================>..] - ETA: 0s - loss: 0.3193 - accuracy: 0.8906
Epoch 19: val_loss did not improve from 0.68804
17/17 [==============================] - 0s 13ms/step - loss: 0.3271 - accuracy: 0.8851 - val_loss: 0.6882 - val_accuracy: 0.5092
Epoch 20/100
16/17 [===========================>..] - ETA: 0s - loss: 0.3006 - accuracy: 0.9004
Epoch 20: val_loss did not improve from 0.68804
17/17 [==============================] - 0s 12ms/step - loss: 0.3008 - accuracy: 0.9023 - val_loss: 0.6888 - val_accuracy: 0.5215
Epoch 21/100
17/17 [==============================] - ETA: 0s - loss: 0.2916 - accuracy: 0.8985
Epoch 21: val_loss did not improve from 0.68804
17/17 [==============================] - 0s 13ms/step - loss: 0.2916 - accuracy: 0.8985 - val_loss: 0.6913 - val_accuracy: 0.5276
Epoch 22/100
13/17 [=====================>........] - ETA: 0s - loss: 0.2745 - accuracy: 0.9014
Epoch 22: val_loss did not improve from 0.68804
17/17 [===========================

...

Epoch 15/100
19/24 [======================>.......] - ETA: 0s - loss: 0.4545 - accuracy: 0.8043
Epoch 15: val_loss did not improve from 0.70991
24/24 [==============================] - 0s 10ms/step - loss: 0.4519 - accuracy: 0.8016 - val_loss: 0.8502 - val_accuracy: 0.4259
Epoch 16/100
19/24 [======================>.......] - ETA: 0s - loss: 0.4166 - accuracy: 0.8191Restoring model weights from the end of the best epoch: 1.

Epoch 16: val_loss did not improve from 0.70991
24/24 [==============================] - 0s 11ms/step - loss: 0.4318 - accuracy: 0.8083 - val_loss: 0.8450 - val_accuracy: 0.4259
Epoch 16: early stopping
3/3 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_Full_text_S_label_layer_3.png
Test Accuracy for Layer 3: 0.4348
Test AUC for Layer 3: 0.4248

Average Test Accuracy across all layers: 0.4979
Average Test AUC across all layers: 0.4925

Training MLP model for Full text and L_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Va

17/17 [==============================] - 0s 11ms/step - loss: 0.4391 - accuracy: 0.7989 - val_loss: 0.6959 - val_accuracy: 0.4908
Epoch 17/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3956 - accuracy: 0.8005
Epoch 17: val_loss did not improve from 0.69501
17/17 [==============================] - 0s 11ms/step - loss: 0.3849 - accuracy: 0.8142 - val_loss: 0.6966 - val_accuracy: 0.5092
Epoch 18/100
12/17 [====================>.........] - ETA: 0s - loss: 0.3888 - accuracy: 0.8125
Epoch 18: val_loss did not improve from 0.69501
17/17 [==============================] - 0s 12ms/step - loss: 0.3905 - accuracy: 0.8142 - val_loss: 0.6980 - val_accuracy: 0.5276
Epoch 19/100
17/17 [==============================] - ETA: 0s - loss: 0.3427 - accuracy: 0.8602
Epoch 19: val_loss did not improve from 0.69501
17/17 [==============================] - 0s 12ms/step - loss: 0.3427 - accuracy: 0.8602 - val_loss: 0.7005 - val_accuracy: 0.5153
Epoch 20/100
12/17 [====================>.......

Epoch 7/100
18/22 [=======================>......] - ETA: 0s - loss: 0.5994 - accuracy: 0.6788
Epoch 7: val_loss did not improve from 0.67707
22/22 [==============================] - 0s 11ms/step - loss: 0.5987 - accuracy: 0.6788 - val_loss: 0.6794 - val_accuracy: 0.6364
Epoch 8/100
18/22 [=======================>......] - ETA: 0s - loss: 0.6146 - accuracy: 0.6528
Epoch 8: val_loss did not improve from 0.67707
22/22 [==============================] - 0s 11ms/step - loss: 0.6131 - accuracy: 0.6540 - val_loss: 0.6781 - val_accuracy: 0.6515
Epoch 9/100
19/22 [========================>.....] - ETA: 0s - loss: 0.5989 - accuracy: 0.6678
Epoch 9: val_loss did not improve from 0.67707
22/22 [==============================] - 0s 10ms/step - loss: 0.5995 - accuracy: 0.6715 - val_loss: 0.6780 - val_accuracy: 0.6667
Epoch 10/100
19/22 [========================>.....] - ETA: 0s - loss: 0.5406 - accuracy: 0.7648
Epoch 10: val_loss improved from 0.67707 to 0.67638, saving model to OpenAI_MLP/visualiz

                                                                 
 dense_117 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_88 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_59 (Dropout)        (None, 256)               0         
                                                                 
 dense_118 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_89 (Ba  (None, 128)               512       
 tchNormalization)                                               
                                                                 
 dense_119 (Dense)           (None, 1)                 129       
                                                                 
Total para

Epoch 21/100
24/24 [==============================] - ETA: 0s - loss: 0.3477 - accuracy: 0.8535
Epoch 21: val_loss did not improve from 0.62444
24/24 [==============================] - 0s 11ms/step - loss: 0.3477 - accuracy: 0.8535 - val_loss: 0.6319 - val_accuracy: 0.6481
Epoch 22/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3499 - accuracy: 0.8602
Epoch 22: val_loss did not improve from 0.62444
24/24 [==============================] - 0s 10ms/step - loss: 0.3502 - accuracy: 0.8628 - val_loss: 0.6391 - val_accuracy: 0.6296
Epoch 23/100
24/24 [==============================] - ETA: 0s - loss: 0.3695 - accuracy: 0.8469
Epoch 23: val_loss did not improve from 0.62444
24/24 [==============================] - 0s 10ms/step - loss: 0.3695 - accuracy: 0.8469 - val_loss: 0.6554 - val_accuracy: 0.6296
Epoch 24/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3335 - accuracy: 0.8701
Epoch 24: val_loss did not improve from 0.62444
24/24 [===========================

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/SLB'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/SLB', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (SLB)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/SLB', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/SLB', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/SLB', 'Merged_OpenAI_MLP/visualizations_summary/SLB']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_SLB_completed_openai.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {1: 472, 0: 457}
Class distribution for long-term prediction: {1: 494, 0: 435}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 

 dense_124 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_93 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_62 (Dropout)        (None, 512)               0         
                                                                 
 dense_125 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_94 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_63 (Dropout)        (None, 256)               0         
                                                                 
 dense_126 (Dense)           (None, 128)               32896     
          

 dense_131 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
21/24 [=========================>....] - ETA: 0s - loss: 0.8845 - accuracy: 0.5104
Epoch 1: val_loss improved from inf to 0.69025, saving model to Merged_OpenAI_MLP/visualizations_mlp/SLB\best_mlp_merged_S_label_layer_3.weights.h5
24/24 [==============================] - 2s 20ms/step - loss: 0.8863 - accuracy: 0.5087 - val_loss: 0.6902 - val_accuracy: 0.5741
Epoch 2/100
21/24 [=========================>....] - ETA: 0s - loss: 0.7837 - accuracy: 0.5461
Epoch 2: val_loss improved from 0.69025 to 0.68817, saving model to Merged_OpenAI_MLP/visualizations_mlp/SLB\best_mlp_merged_S_label_layer_3.weights.h5
24/24 [==============================] - 0s 14ms/step - l

 dropout_67 (Dropout)        (None, 256)               0         
                                                                 
 dense_134 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_101 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_135 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
16/17 [===========================>..] - ETA: 0s - loss: 0.9931 - accuracy: 0.4492
Epoch 1: val_loss improved from inf to 0.69863, saving model to Merged_OpenAI_MLP/visualizations_mlp/SLB\best_mlp_merged

Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.7706 - accuracy: 0.5708
Epoch 2: val_loss did not improve from 0.69897
22/22 [==============================] - 0s 12ms/step - loss: 0.7706 - accuracy: 0.5708 - val_loss: 0.7000 - val_accuracy: 0.3485
Epoch 3/100
21/22 [===========================>..] - ETA: 0s - loss: 0.7373 - accuracy: 0.5863
Epoch 3: val_loss did not improve from 0.69897
22/22 [==============================] - 0s 12ms/step - loss: 0.7389 - accuracy: 0.5854 - val_loss: 0.6992 - val_accuracy: 0.3485
Epoch 4/100
21/22 [===========================>..] - ETA: 0s - loss: 0.7327 - accuracy: 0.5893
Epoch 4: val_loss improved from 0.69897 to 0.69893, saving model to Merged_OpenAI_MLP/visualizations_mlp/SLB\best_mlp_merged_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 13ms/step - loss: 0.7286 - accuracy: 0.5912 - val_loss: 0.6989 - val_accuracy: 0.3333
Epoch 5/100
21/22 [===========================>..] - ETA: 0s - loss: 0.6716 - a

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_140 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_105 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_70 (Dropout)        (None, 512)               0         
                                                                 
 dense_141 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_106 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_71 (Dropout)        (None, 256)               0         
          

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP/visualizations_mlp/MPC'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP/visualizations_summary/MPC', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS （MPC)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/MPC', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        mlp_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/MPC', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP/visualizations_mlp/MPC',
                      'OpenAI_MLP/visualizations_summary/MPC']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_MPC_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {1: 482, 0: 447}
Class distribution for long-term prediction: {1: 555, 0: 374}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 d

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_148 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_111 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_74 (Dropout)        (None, 512)               0         
                                                                 
 dense_149 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_112 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_75 (Dropout)        (None, 256)               0         
          

 batch_normalization_116 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_155 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
21/24 [=========================>....] - ETA: 0s - loss: 0.9116 - accuracy: 0.4613
Epoch 1: val_loss improved from inf to 0.68974, saving model to OpenAI_MLP/visualizations_mlp/MPC\best_mlp_title_S_label_layer_3.weights.h5
24/24 [==============================] - 2s 20ms/step - loss: 0.9032 - accuracy: 0.4674 - val_loss: 0.6897 - val_accuracy: 0.5833
Epoch 2/100
22/24 [==========================>...] - ETA: 0s - loss: 0.8405 - accuracy: 0.5298
Epoch 2: val_loss i

Epoch 26/100
24/24 [==============================] - ETA: 0s - loss: 0.2499 - accuracy: 0.9148
Epoch 26: val_loss did not improve from 0.68061
24/24 [==============================] - 0s 11ms/step - loss: 0.2499 - accuracy: 0.9148 - val_loss: 0.7167 - val_accuracy: 0.5556
Epoch 27/100
19/24 [======================>.......] - ETA: 0s - loss: 0.2584 - accuracy: 0.9128
Epoch 27: val_loss did not improve from 0.68061
24/24 [==============================] - 0s 11ms/step - loss: 0.2581 - accuracy: 0.9121 - val_loss: 0.7218 - val_accuracy: 0.5556
Epoch 28/100
24/24 [==============================] - ETA: 0s - loss: 0.2520 - accuracy: 0.9174
Epoch 28: val_loss did not improve from 0.68061
24/24 [==============================] - 0s 11ms/step - loss: 0.2520 - accuracy: 0.9174 - val_loss: 0.7362 - val_accuracy: 0.5370
Epoch 29/100
19/24 [======================>.......] - ETA: 0s - loss: 0.2317 - accuracy: 0.9194
Epoch 29: val_loss did not improve from 0.68061
24/24 [===========================

17/17 [==============================] - 0s 13ms/step - loss: 0.4854 - accuracy: 0.7701 - val_loss: 1.0224 - val_accuracy: 0.3926
Epoch 13/100
13/17 [=====================>........] - ETA: 0s - loss: 0.4265 - accuracy: 0.8077
Epoch 13: val_loss did not improve from 0.72510
17/17 [==============================] - 0s 12ms/step - loss: 0.4178 - accuracy: 0.8180 - val_loss: 1.0254 - val_accuracy: 0.3926
Epoch 14/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3977 - accuracy: 0.8125
Epoch 14: val_loss did not improve from 0.72510
17/17 [==============================] - 0s 12ms/step - loss: 0.3998 - accuracy: 0.8123 - val_loss: 1.0354 - val_accuracy: 0.3926
Epoch 15/100
13/17 [=====================>........] - ETA: 0s - loss: 0.3728 - accuracy: 0.8462
Epoch 15: val_loss did not improve from 0.72510
17/17 [==============================] - 0s 11ms/step - loss: 0.3830 - accuracy: 0.8352 - val_loss: 1.0353 - val_accuracy: 0.3926
Epoch 16/100
17/17 [============================

...

Epoch 18/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3898 - accuracy: 0.8273
Epoch 18: val_loss did not improve from 0.68188
24/24 [==============================] - 0s 11ms/step - loss: 0.3882 - accuracy: 0.8309 - val_loss: 0.6997 - val_accuracy: 0.5648
Epoch 19/100
22/24 [==========================>...] - ETA: 0s - loss: 0.4070 - accuracy: 0.8239
Epoch 19: val_loss did not improve from 0.68188
24/24 [==============================] - 0s 12ms/step - loss: 0.4074 - accuracy: 0.8256 - val_loss: 0.7019 - val_accuracy: 0.5370
Epoch 20/100
24/24 [==============================] - ETA: 0s - loss: 0.3745 - accuracy: 0.8469
Epoch 20: val_loss did not improve from 0.68188
24/24 [==============================] - 0s 11ms/step - loss: 0.3745 - accuracy: 0.8469 - val_loss: 0.7061 - val_accuracy: 0.5278
Epoch 21/100
23/24 [===========================>..] - ETA: 0s - loss: 0.3640 - accuracy: 0.8478
Epoch 21: val_loss did not improve from 0.68188
24/24 [===========================

17/17 [==============================] - 0s 15ms/step - loss: 0.5036 - accuracy: 0.7625 - val_loss: 0.6846 - val_accuracy: 0.6012
Epoch 12/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4923 - accuracy: 0.7695
Epoch 12: val_loss improved from 0.68463 to 0.68367, saving model to OpenAI_MLP/visualizations_mlp/MPC\best_mlp_full_text_L_label_layer_1.weights.h5
17/17 [==============================] - 0s 16ms/step - loss: 0.4905 - accuracy: 0.7682 - val_loss: 0.6837 - val_accuracy: 0.6074
Epoch 13/100
12/17 [====================>.........] - ETA: 0s - loss: 0.4731 - accuracy: 0.7839
Epoch 13: val_loss did not improve from 0.68367
17/17 [==============================] - 0s 12ms/step - loss: 0.4658 - accuracy: 0.7797 - val_loss: 0.6846 - val_accuracy: 0.5767
Epoch 14/100
12/17 [====================>.........] - ETA: 0s - loss: 0.4592 - accuracy: 0.7682
Epoch 14: val_loss did not improve from 0.68367
17/17 [==============================] - 0s 12ms/step - loss: 0.4551 - accura

Epoch 5/100
19/22 [========================>.....] - ETA: 0s - loss: 0.6587 - accuracy: 0.6316
Epoch 5: val_loss did not improve from 0.80384
22/22 [==============================] - 0s 10ms/step - loss: 0.6628 - accuracy: 0.6234 - val_loss: 1.1372 - val_accuracy: 0.1061
Epoch 6/100
19/22 [========================>.....] - ETA: 0s - loss: 0.6377 - accuracy: 0.6382
Epoch 6: val_loss did not improve from 0.80384
22/22 [==============================] - 0s 10ms/step - loss: 0.6333 - accuracy: 0.6409 - val_loss: 1.1900 - val_accuracy: 0.1061
Epoch 7/100
19/22 [========================>.....] - ETA: 0s - loss: 0.6082 - accuracy: 0.6760
Epoch 7: val_loss did not improve from 0.80384
22/22 [==============================] - 0s 11ms/step - loss: 0.6112 - accuracy: 0.6759 - val_loss: 1.2226 - val_accuracy: 0.1061
Epoch 8/100
18/22 [=======================>......] - ETA: 0s - loss: 0.5853 - accuracy: 0.6944
Epoch 8: val_loss did not improve from 0.80384
22/22 [==============================] - 0

24/24 [==============================] - 0s 13ms/step - loss: 0.6519 - accuracy: 0.6431 - val_loss: 0.6160 - val_accuracy: 0.7315
Epoch 8/100
22/24 [==========================>...] - ETA: 0s - loss: 0.5556 - accuracy: 0.7287
Epoch 8: val_loss improved from 0.61595 to 0.60772, saving model to OpenAI_MLP/visualizations_mlp/MPC\best_mlp_full_text_L_label_layer_3.weights.h5
24/24 [==============================] - 0s 13ms/step - loss: 0.5596 - accuracy: 0.7244 - val_loss: 0.6077 - val_accuracy: 0.7315
Epoch 9/100
22/24 [==========================>...] - ETA: 0s - loss: 0.5736 - accuracy: 0.7045
Epoch 9: val_loss improved from 0.60772 to 0.60137, saving model to OpenAI_MLP/visualizations_mlp/MPC\best_mlp_full_text_L_label_layer_3.weights.h5
24/24 [==============================] - 0s 13ms/step - loss: 0.5824 - accuracy: 0.6977 - val_loss: 0.6014 - val_accuracy: 0.7315
Epoch 10/100
23/24 [===========================>..] - ETA: 0s - loss: 0.5644 - accuracy: 0.7052
Epoch 10: val_loss improved 

19/24 [======================>.......] - ETA: 0s - loss: 0.2301 - accuracy: 0.9293
Epoch 33: val_loss did not improve from 0.57782
24/24 [==============================] - 0s 11ms/step - loss: 0.2363 - accuracy: 0.9268 - val_loss: 0.6129 - val_accuracy: 0.6759
Epoch 34/100
19/24 [======================>.......] - ETA: 0s - loss: 0.2117 - accuracy: 0.9293
Epoch 34: val_loss did not improve from 0.57782
24/24 [==============================] - 0s 10ms/step - loss: 0.2089 - accuracy: 0.9294 - val_loss: 0.6374 - val_accuracy: 0.6944
Epoch 35/100
24/24 [==============================] - ETA: 0s - loss: 0.2015 - accuracy: 0.9361
Epoch 35: val_loss did not improve from 0.57782
24/24 [==============================] - 0s 10ms/step - loss: 0.2015 - accuracy: 0.9361 - val_loss: 0.6540 - val_accuracy: 0.7037
Epoch 36/100
24/24 [==============================] - ETA: 0s - loss: 0.1980 - accuracy: 0.9401
Epoch 36: val_loss did not improve from 0.57782
24/24 [==============================] - 0s 11m

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/MPC'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/MPC', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (MPC)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/MPC', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/MPC', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/MPC', 'Merged_OpenAI_MLP/visualizations_summary/MPC']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_MPC_completed_openai.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {1: 482, 0: 447}
Class distribution for long-term prediction: {1: 555, 0: 374}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_192 

 dense_196 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_147 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_98 (Dropout)        (None, 512)               0         
                                                                 
 dense_197 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_148 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_99 (Dropout)        (None, 256)               0         
                                                                 
 dense_198 (Dense)           (None, 128)               32896     
          

                                                                 
 dense_202 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_152 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_203 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
21/24 [=========================>....] - ETA: 0s - loss: 0.8718 - accuracy: 0.4970
Epoch 1: val_loss improved from inf to 0.71687, saving model to Merged_OpenAI_MLP/visualizations_mlp/MPC\best_mlp_merged_S_label_layer_3.weights.h5
24/24 [==============================]

17/17 [==============================] - 2s 25ms/step - loss: 0.8377 - accuracy: 0.5536 - val_loss: 0.7002 - val_accuracy: 0.3926
Epoch 2/100
16/17 [===========================>..] - ETA: 0s - loss: 0.8110 - accuracy: 0.5645
Epoch 2: val_loss did not improve from 0.70022
17/17 [==============================] - 0s 13ms/step - loss: 0.8052 - accuracy: 0.5690 - val_loss: 0.7062 - val_accuracy: 0.3926
Epoch 3/100
17/17 [==============================] - ETA: 0s - loss: 0.7124 - accuracy: 0.6513
Epoch 3: val_loss did not improve from 0.70022
17/17 [==============================] - 0s 13ms/step - loss: 0.7124 - accuracy: 0.6513 - val_loss: 0.7109 - val_accuracy: 0.3926
Epoch 4/100
17/17 [==============================] - ETA: 0s - loss: 0.7211 - accuracy: 0.5920
Epoch 4: val_loss did not improve from 0.70022
17/17 [==============================] - 0s 12ms/step - loss: 0.7211 - accuracy: 0.5920 - val_loss: 0.7161 - val_accuracy: 0.3926
Epoch 5/100
13/17 [=====================>........] - E

Epoch 5/100
22/22 [==============================] - ETA: 0s - loss: 0.6707 - accuracy: 0.6380
Epoch 5: val_loss improved from 0.61180 to 0.59420, saving model to Merged_OpenAI_MLP/visualizations_mlp/MPC\best_mlp_merged_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 14ms/step - loss: 0.6707 - accuracy: 0.6380 - val_loss: 0.5942 - val_accuracy: 0.8939
Epoch 6/100
17/22 [======================>.......] - ETA: 0s - loss: 0.6310 - accuracy: 0.6673
Epoch 6: val_loss improved from 0.59420 to 0.58512, saving model to Merged_OpenAI_MLP/visualizations_mlp/MPC\best_mlp_merged_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 13ms/step - loss: 0.6195 - accuracy: 0.6686 - val_loss: 0.5851 - val_accuracy: 0.8939
Epoch 7/100
17/22 [======================>.......] - ETA: 0s - loss: 0.6191 - accuracy: 0.6728
Epoch 7: val_loss improved from 0.58512 to 0.57156, saving model to Merged_OpenAI_MLP/visualizations_mlp/MPC\best_mlp_merged_L_label_layer_2.weights.h5

Epoch 30/100
17/22 [======================>.......] - ETA: 0s - loss: 0.2811 - accuracy: 0.8915
Epoch 30: val_loss did not improve from 0.52487
22/22 [==============================] - 0s 11ms/step - loss: 0.2807 - accuracy: 0.8920 - val_loss: 0.6671 - val_accuracy: 0.7273
Epoch 31/100
21/22 [===========================>..] - ETA: 0s - loss: 0.2501 - accuracy: 0.9196
Epoch 31: val_loss did not improve from 0.52487
22/22 [==============================] - 0s 12ms/step - loss: 0.2491 - accuracy: 0.9197 - val_loss: 0.7146 - val_accuracy: 0.6818
Epoch 32/100
17/22 [======================>.......] - ETA: 0s - loss: 0.2426 - accuracy: 0.9283
Epoch 32: val_loss did not improve from 0.52487
22/22 [==============================] - 0s 12ms/step - loss: 0.2403 - accuracy: 0.9285 - val_loss: 0.7359 - val_accuracy: 0.6818
Epoch 33/100
18/22 [=======================>......] - ETA: 0s - loss: 0.2231 - accuracy: 0.9167
Epoch 33: val_loss did not improve from 0.52487
22/22 [===========================

24/24 [==============================] - 0s 11ms/step - loss: 0.4857 - accuracy: 0.7710 - val_loss: 0.7506 - val_accuracy: 0.3056
Epoch 14/100
24/24 [==============================] - ETA: 0s - loss: 0.4306 - accuracy: 0.8069
Epoch 14: val_loss did not improve from 0.70554
24/24 [==============================] - 0s 11ms/step - loss: 0.4306 - accuracy: 0.8069 - val_loss: 0.7354 - val_accuracy: 0.3889
Epoch 15/100
19/24 [======================>.......] - ETA: 0s - loss: 0.4627 - accuracy: 0.7763
Epoch 15: val_loss did not improve from 0.70554
24/24 [==============================] - 0s 10ms/step - loss: 0.4647 - accuracy: 0.7763 - val_loss: 0.7296 - val_accuracy: 0.4167
Epoch 16/100
23/24 [===========================>..] - ETA: 0s - loss: 0.4259 - accuracy: 0.8030Restoring model weights from the end of the best epoch: 1.

Epoch 16: val_loss did not improve from 0.70554
24/24 [==============================] - 0s 11ms/step - loss: 0.4278 - accuracy: 0.8029 - val_loss: 0.7268 - val_accura

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP/visualizations_mlp/CVX'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP/visualizations_summary/CVX', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS （CVX)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/CVX', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        mlp_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/CVX', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP/visualizations_mlp/CVX',
                      'OpenAI_MLP/visualizations_summary/CVX']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_CVX_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {1: 499, 0: 430}
Class distribution for long-term prediction: {1: 521, 0: 408}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 d

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_220 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_165 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_110 (Dropout)       (None, 512)               0         
                                                                 
 dense_221 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_166 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_111 (Dropout)       (None, 256)               0         
          

22/22 [==============================] - 0s 11ms/step - loss: 0.3066 - accuracy: 0.8861 - val_loss: 0.6788 - val_accuracy: 0.5909
Epoch 22/100
17/22 [======================>.......] - ETA: 0s - loss: 0.3301 - accuracy: 0.8658
Epoch 22: val_loss did not improve from 0.67482
22/22 [==============================] - 0s 12ms/step - loss: 0.3257 - accuracy: 0.8672 - val_loss: 0.6806 - val_accuracy: 0.5455
Epoch 23/100
17/22 [======================>.......] - ETA: 0s - loss: 0.2897 - accuracy: 0.9007
Epoch 23: val_loss did not improve from 0.67482
22/22 [==============================] - 0s 11ms/step - loss: 0.2920 - accuracy: 0.8978 - val_loss: 0.6892 - val_accuracy: 0.5303
Epoch 24/100
18/22 [=======================>......] - ETA: 0s - loss: 0.2832 - accuracy: 0.9080
Epoch 24: val_loss did not improve from 0.67482
22/22 [==============================] - 0s 11ms/step - loss: 0.2827 - accuracy: 0.9036 - val_loss: 0.6966 - val_accuracy: 0.5303
Epoch 25/100
18/22 [=======================>....

Epoch 7/100
23/24 [===========================>..] - ETA: 0s - loss: 0.6282 - accuracy: 0.6780
Epoch 7: val_loss did not improve from 0.67472
24/24 [==============================] - 0s 11ms/step - loss: 0.6242 - accuracy: 0.6791 - val_loss: 0.6867 - val_accuracy: 0.6019
Epoch 8/100
22/24 [==========================>...] - ETA: 0s - loss: 0.5898 - accuracy: 0.6960
Epoch 8: val_loss did not improve from 0.67472
24/24 [==============================] - 0s 11ms/step - loss: 0.5820 - accuracy: 0.7017 - val_loss: 0.6928 - val_accuracy: 0.6019
Epoch 9/100
24/24 [==============================] - ETA: 0s - loss: 0.5530 - accuracy: 0.7137
Epoch 9: val_loss did not improve from 0.67472
24/24 [==============================] - 0s 11ms/step - loss: 0.5530 - accuracy: 0.7137 - val_loss: 0.7001 - val_accuracy: 0.6019
Epoch 10/100
24/24 [==============================] - ETA: 0s - loss: 0.5688 - accuracy: 0.6951
Epoch 10: val_loss did not improve from 0.67472
24/24 [==============================] -

Epoch 6/100
17/17 [==============================] - ETA: 0s - loss: 0.6286 - accuracy: 0.6590
Epoch 6: val_loss improved from 0.66234 to 0.66172, saving model to OpenAI_MLP/visualizations_mlp/CVX\best_mlp_title_L_label_layer_1.weights.h5
17/17 [==============================] - 0s 15ms/step - loss: 0.6286 - accuracy: 0.6590 - val_loss: 0.6617 - val_accuracy: 0.6258
Epoch 7/100
16/17 [===========================>..] - ETA: 0s - loss: 0.5739 - accuracy: 0.7129
Epoch 7: val_loss did not improve from 0.66172
17/17 [==============================] - 0s 13ms/step - loss: 0.5723 - accuracy: 0.7146 - val_loss: 0.6634 - val_accuracy: 0.6258
Epoch 8/100
17/17 [==============================] - ETA: 0s - loss: 0.5515 - accuracy: 0.7222
Epoch 8: val_loss did not improve from 0.66172
17/17 [==============================] - 0s 12ms/step - loss: 0.5515 - accuracy: 0.7222 - val_loss: 0.6671 - val_accuracy: 0.6258
Epoch 9/100
13/17 [=====================>........] - ETA: 0s - loss: 0.5261 - accuracy:

...

Epoch 3/100
23/24 [===========================>..] - ETA: 0s - loss: 0.7758 - accuracy: 0.5584
Epoch 3: val_loss improved from 0.68347 to 0.68090, saving model to OpenAI_MLP/visualizations_mlp/CVX\best_mlp_full_text_S_label_layer_3.weights.h5
24/24 [==============================] - 0s 13ms/step - loss: 0.7756 - accuracy: 0.5606 - val_loss: 0.6809 - val_accuracy: 0.6019
Epoch 4/100
23/24 [===========================>..] - ETA: 0s - loss: 0.6963 - accuracy: 0.6277
Epoch 4: val_loss improved from 0.68090 to 0.67937, saving model to OpenAI_MLP/visualizations_mlp/CVX\best_mlp_full_text_S_label_layer_3.weights.h5
24/24 [==============================] - 0s 13ms/step - loss: 0.6980 - accuracy: 0.6258 - val_loss: 0.6794 - val_accuracy: 0.6019
Epoch 5/100
21/24 [=========================>....] - ETA: 0s - loss: 0.6636 - accuracy: 0.6384
Epoch 5: val_loss improved from 0.67937 to 0.67801, saving model to OpenAI_MLP/visualizations_mlp/CVX\best_mlp_full_text_S_label_layer_3.weights.h5
24/24 [====

Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
16/17 [===========================>..] - ETA: 0s - loss: 0.8772 - accuracy: 0.5117
Epoch 1: val_loss improved from inf to 0.69021, saving model to OpenAI_MLP/visualizations_mlp/CVX\best_mlp_full_text_L_label_layer_1.weights.h5
17/17 [==============================] - 2s 25ms/step - loss: 0.8787 - accuracy: 0.5096 - val_loss: 0.6902 - val_accuracy: 0.6196
Epoch 2/100
16/17 [===========================>..] - ETA: 0s - loss: 0.7736 - accuracy: 0.5293
Epoch 2: val_loss improved from 0.69021 to 0.68542, saving model to OpenAI_MLP/visualizations_mlp/CVX\best_mlp_full_text_L_label_layer_1.weights.h5
17/17 [==============================] - 0s 16ms/step - loss: 0.7737 - accuracy: 0.5326 - val_loss: 0.6854 - val_accuracy: 0.6258
Epoch 3/100
16/17 [===========================>..] - ETA: 0s - loss

12/17 [====================>.........] - ETA: 0s - loss: 0.3125 - accuracy: 0.8880
Epoch 26: val_loss did not improve from 0.67537
17/17 [==============================] - 0s 11ms/step - loss: 0.3097 - accuracy: 0.8851 - val_loss: 0.7011 - val_accuracy: 0.4908
Epoch 27/100
17/17 [==============================] - ETA: 0s - loss: 0.3101 - accuracy: 0.8793
Epoch 27: val_loss did not improve from 0.67537
17/17 [==============================] - 0s 12ms/step - loss: 0.3101 - accuracy: 0.8793 - val_loss: 0.7067 - val_accuracy: 0.4785
Epoch 28/100
12/17 [====================>.........] - ETA: 0s - loss: 0.2789 - accuracy: 0.9036Restoring model weights from the end of the best epoch: 13.

Epoch 28: val_loss did not improve from 0.67537
17/17 [==============================] - 0s 12ms/step - loss: 0.2731 - accuracy: 0.9100 - val_loss: 0.7116 - val_accuracy: 0.4724
Epoch 28: early stopping
3/3 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_Full_text_L_label_laye

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_260 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_195 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_130 (Dropout)       (None, 512)               0         
                                                                 
 dense_261 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_196 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_131 (Dropout)       (None, 256)               0         
          

24/24 [==============================] - 0s 15ms/step - loss: 0.4024 - accuracy: 0.8189 - val_loss: 0.6543 - val_accuracy: 0.6481
Epoch 20/100
22/24 [==========================>...] - ETA: 0s - loss: 0.3921 - accuracy: 0.8267
Epoch 20: val_loss did not improve from 0.65137
24/24 [==============================] - 0s 12ms/step - loss: 0.3939 - accuracy: 0.8242 - val_loss: 0.6566 - val_accuracy: 0.6389
Epoch 21/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3579 - accuracy: 0.8684
Epoch 21: val_loss did not improve from 0.65137
24/24 [==============================] - 0s 11ms/step - loss: 0.3570 - accuracy: 0.8628 - val_loss: 0.6657 - val_accuracy: 0.6389
Epoch 22/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3278 - accuracy: 0.8734
Epoch 22: val_loss did not improve from 0.65137
24/24 [==============================] - 0s 11ms/step - loss: 0.3323 - accuracy: 0.8708 - val_loss: 0.6643 - val_accuracy: 0.6296
Epoch 23/100
21/24 [=========================>..

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/CVX'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/CVX', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (CVX)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/CVX', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/CVX', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/CVX', 'Merged_OpenAI_MLP/visualizations_summary/CVX']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_CVX_completed_openai.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {1: 499, 0: 430}
Class distribution for long-term prediction: {1: 521, 0: 408}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_66"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_264 

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_268 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_201 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_134 (Dropout)       (None, 512)               0         
                                                                 
 dense_269 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_202 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_135 (Dropout)       (None, 256)               0         
          

 batch_normalization_206 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_275 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
20/24 [========================>.....] - ETA: 0s - loss: 0.8760 - accuracy: 0.5031
Epoch 1: val_loss improved from inf to 0.67975, saving model to Merged_OpenAI_MLP/visualizations_mlp/CVX\best_mlp_merged_S_label_layer_3.weights.h5
24/24 [==============================] - 2s 20ms/step - loss: 0.8711 - accuracy: 0.4993 - val_loss: 0.6798 - val_accuracy: 0.6019
Epoch 2/100
21/24 [=========================>....] - ETA: 0s - loss: 0.8265 - accuracy: 0.5060
Epoch 2: va

 dense_279 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
16/17 [===========================>..] - ETA: 0s - loss: 0.9022 - accuracy: 0.5098
Epoch 1: val_loss improved from inf to 0.67585, saving model to Merged_OpenAI_MLP/visualizations_mlp/CVX\best_mlp_merged_L_label_layer_1.weights.h5
17/17 [==============================] - 2s 25ms/step - loss: 0.9074 - accuracy: 0.5057 - val_loss: 0.6758 - val_accuracy: 0.6258
Epoch 2/100
16/17 [===========================>..] - ETA: 0s - loss: 0.7990 - accuracy: 0.5840
Epoch 2: val_loss improved from 0.67585 to 0.66680, saving model to Merged_OpenAI_MLP/visualizations_mlp/CVX\best_mlp_merged_L_label_layer_1.weights.h5
17/17 [==============================] - 0s 16ms/step - l

17/17 [==============================] - ETA: 0s - loss: 0.2943 - accuracy: 0.8831
Epoch 24: val_loss did not improve from 0.65417
17/17 [==============================] - 0s 12ms/step - loss: 0.2943 - accuracy: 0.8831 - val_loss: 0.6616 - val_accuracy: 0.5890
Epoch 25/100
17/17 [==============================] - ETA: 0s - loss: 0.2905 - accuracy: 0.8946
Epoch 25: val_loss did not improve from 0.65417
17/17 [==============================] - 0s 14ms/step - loss: 0.2905 - accuracy: 0.8946 - val_loss: 0.6609 - val_accuracy: 0.5951
Epoch 26/100
17/17 [==============================] - ETA: 0s - loss: 0.2616 - accuracy: 0.9023
Epoch 26: val_loss did not improve from 0.65417
17/17 [==============================] - 0s 12ms/step - loss: 0.2616 - accuracy: 0.9023 - val_loss: 0.6600 - val_accuracy: 0.6074
Epoch 27/100
12/17 [====================>.........] - ETA: 0s - loss: 0.2561 - accuracy: 0.9271
Epoch 27: val_loss did not improve from 0.65417
17/17 [==============================] - 0s 11m

22/22 [==============================] - 0s 12ms/step - loss: 0.4364 - accuracy: 0.8117 - val_loss: 1.1720 - val_accuracy: 0.2121
Epoch 16/100
18/22 [=======================>......] - ETA: 0s - loss: 0.4415 - accuracy: 0.7951Restoring model weights from the end of the best epoch: 1.

Epoch 16: val_loss did not improve from 0.74546
22/22 [==============================] - 0s 11ms/step - loss: 0.4402 - accuracy: 0.7956 - val_loss: 1.1890 - val_accuracy: 0.2121
Epoch 16: early stopping
4/4 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_merged_L_label_layer_2.png
Test Accuracy for Layer 2: 0.3704
Test AUC for Layer 2: 0.4485

Layer 3:
Training period: 01/01/2019 - 31/05/2022
Validation period: 01/06/2022 - 31/12/2022
Testing period: 01/01/2023 - 31/05/2023
Training data: 751 samples
Validation data: 108 samples
Test data: 69 samples
Embeddings shapes - Train: (751, 1536), Val: (108, 1536), Test: (69, 1536)
Creating MLP model with input shape: (1536,)
MLP mo

Epoch 17/100
22/24 [==========================>...] - ETA: 0s - loss: 0.4173 - accuracy: 0.8224
Epoch 17: val_loss did not improve from 0.67316
24/24 [==============================] - 0s 12ms/step - loss: 0.4153 - accuracy: 0.8242 - val_loss: 0.6828 - val_accuracy: 0.5926
Epoch 18/100
23/24 [===========================>..] - ETA: 0s - loss: 0.3827 - accuracy: 0.8370
Epoch 18: val_loss did not improve from 0.67316
24/24 [==============================] - 0s 11ms/step - loss: 0.3830 - accuracy: 0.8375 - val_loss: 0.6867 - val_accuracy: 0.5833
Epoch 19/100
23/24 [===========================>..] - ETA: 0s - loss: 0.3956 - accuracy: 0.8261
Epoch 19: val_loss did not improve from 0.67316
24/24 [==============================] - 0s 11ms/step - loss: 0.3951 - accuracy: 0.8256 - val_loss: 0.6973 - val_accuracy: 0.5463
Epoch 20/100
23/24 [===========================>..] - ETA: 0s - loss: 0.3842 - accuracy: 0.8438
Epoch 20: val_loss did not improve from 0.67316
24/24 [===========================

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from xgboost import callback
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, model_type, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.model_type = model_type
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class ClimateNewsOpenAIPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'OpenAI_MLP/visualizations_mlp/COP'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('OpenAI_MLP/visualizations_summary/COP', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert embeddings to numpy arrays
        self.data['Title_embedding'] = self.data['Title_embedding_vector'].apply(parse_embedding)
        self.data['Fulltext_embedding'] = self.data['Full_text_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_title_embedding = self.data['Title_embedding'].iloc[0]
        sample_fulltext_embedding = self.data['Fulltext_embedding'].iloc[0]
        
        print(f"Sample Title embedding shape: {sample_title_embedding.shape}")
        print(f"Sample Fulltext embedding shape: {sample_fulltext_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, text_col, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            text_col: The column containing the embeddings ('Title_embedding' or 'Fulltext_embedding')
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data[text_col].values)
        X_val = np.stack(val_data[text_col].values)
        X_test = np.stack(test_data[text_col].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, model_type, text_col, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            model_type: Model type (MLP)
            text_col: Text column used
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: {model_type} with {display_text} ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_{model_type.lower()}_{display_text.replace(' ', '_')}_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, text_col, label_col, model_type):
        """
        Train and evaluate a model for a specific text column and label column.
        
        Args:
            text_col: The embedding column to use ('Title_embedding' or 'Fulltext_embedding')
            label_col: The label column to use ('S_label' or 'L_label')
            model_type: The model type to use ('MLP')
        """
        # Store results
        display_text = 'Title' if 'Title' in text_col else 'Full text'
        combination_key = f"{model_type}|{display_text}|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training {model_type} model for {display_text} and {label_col}")
        print(f"{'='*80}")
        
        visualization_dir = self.mlp_viz_dir if model_type == 'MLP' else self.xgb_viz_dir
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, text_col, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            if model_type == 'MLP':
                # MLP model training
                model = self.create_mlp_model((1536,))
                print(f"Created MLP model for {display_text} ({label_col})")
                model.summary()
                
                # Setup callbacks
                plot_callback = PlotLearningCallback(model_type, display_text, label_col, i+1, visualization_dir)
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=15,
                    restore_best_weights=True,
                    verbose=1
                )
                model_checkpoint = ModelCheckpoint(
                    filepath=f"{visualization_dir}/best_{model_type.lower()}_{display_text.lower().replace(' ', '_')}_{label_col}_layer_{i+1}.weights.h5",
                    monitor='val_loss',
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1
                )
                
                # Train model
                print(f"Training MLP model...")
                batch_size = 32  # Larger batch size for embeddings
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, plot_callback],
                    verbose=1
                )
                
                # Evaluate on test set
                y_pred_proba = model.predict(X_test)
                y_pred = (y_pred_proba > 0.5).astype(int)
                
                # Create final learning curve visualization
                self.plot_final_learning_curves(history, model_type, text_col, label_col, i+1, visualization_dir)
                
                # Store training history
                training_history = history.history
                
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'model_type': model_type,
                'accuracy': accuracy,
                'auc': auc,
                'history': training_history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for all combinations of text inputs, label columns, and model types."""
        # Define all combinations
        embedding_cols = ['Title_embedding', 'Fulltext_embedding']
        label_cols = ['S_label', 'L_label']
        model_types = ['MLP']
        
        # Run analysis for each combination
        for model_type in model_types:
            for embedding_col in embedding_cols:
                for label_col in label_cols:
                    self.train_and_evaluate_model(embedding_col, label_col, model_type)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS （COP)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Model': model_type,
                'Text': text_col,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing all model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Split by model type
        mlp_data = df[df['Model'] == 'MLP']
        
        # 1. Performance comparison for MLP
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy and AUC bars for MLP
        x = np.arange(len(mlp_data))
        width = 0.35
        
        plt.bar(x - width/2, mlp_data['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, mlp_data['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('MLP Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with OpenAI Embeddings')
        labels = [f"{row['Text']} + {row['Label']}" for _, row in mlp_data.iterrows()]
        plt.xticks(x, labels, rotation=0, ha='right')
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(mlp_data['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(mlp_data['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(mlp_data['Avg Accuracy'].max(), mlp_data['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/COP', "mlp_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        mlp_layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Model': model_type,
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                if model_type == 'MLP':
                    mlp_layer_data.append(layer_info)
        
        # Create visualizations for each model type
        self._create_model_layer_visualization(mlp_layer_data, 'MLP')
    
    def _create_model_layer_visualization(self, layer_data, model_type):
        """Create layer-specific visualizations for a given model type."""
        if not layer_data:
            print(f"No layer data available for {model_type}")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(14, 10))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title(f'{model_type} Accuracy by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title(f'{model_type} AUC by Model Combination and Layer with OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('OpenAI_MLP/visualizations_summary/COP', f"{model_type.lower()}_layer_performance.png")
        plt.savefig(save_path)
        print(f"{model_type} layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['OpenAI_MLP/visualizations_mlp/COP',
                      'OpenAI_MLP/visualizations_summary/COP']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_COP_completed_openai.csv'
    
    # Initialize the predictor with OpenAI embeddings
    predictor = ClimateNewsOpenAIPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing OpenAI embedding vectors from string format...
Sample Title embedding shape: (1536,)
Sample Fulltext embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {0: 469, 1: 460}
Class distribution for long-term prediction: {1: 504, 0: 425}

Training MLP model for Title and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Title (S_label)
Model: "sequential_72"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 d

 dense_292 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_219 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_146 (Dropout)       (None, 512)               0         
                                                                 
 dense_293 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_220 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_147 (Dropout)       (None, 256)               0         
                                                                 
 dense_294 (Dense)           (None, 128)               32896     
          

 dense_299 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
22/24 [==========================>...] - ETA: 0s - loss: 0.8631 - accuracy: 0.5327
Epoch 1: val_loss improved from inf to 0.69578, saving model to OpenAI_MLP/visualizations_mlp/COP\best_mlp_title_S_label_layer_3.weights.h5
24/24 [==============================] - 2s 20ms/step - loss: 0.8646 - accuracy: 0.5313 - val_loss: 0.6958 - val_accuracy: 0.4722
Epoch 2/100
24/24 [==============================] - ETA: 0s - loss: 0.7707 - accuracy: 0.5646
Epoch 2: val_loss did not improve from 0.69578
24/24 [==============================] - 0s 13ms/step - loss: 0.7707 - accuracy: 0.5646 - val_loss: 0.6998 - val_accuracy: 0.4722
Epoch 3/100
22/24 [=====================

Epoch 3/100
16/17 [===========================>..] - ETA: 0s - loss: 0.7838 - accuracy: 0.5430
Epoch 3: val_loss did not improve from 0.69786
17/17 [==============================] - 0s 13ms/step - loss: 0.7810 - accuracy: 0.5441 - val_loss: 0.7014 - val_accuracy: 0.4479
Epoch 4/100
13/17 [=====================>........] - ETA: 0s - loss: 0.7168 - accuracy: 0.5793
Epoch 4: val_loss did not improve from 0.69786
17/17 [==============================] - 0s 12ms/step - loss: 0.7256 - accuracy: 0.5766 - val_loss: 0.7023 - val_accuracy: 0.4479
Epoch 5/100
12/17 [====================>.........] - ETA: 0s - loss: 0.6841 - accuracy: 0.6120
Epoch 5: val_loss did not improve from 0.69786
17/17 [==============================] - 0s 11ms/step - loss: 0.6566 - accuracy: 0.6322 - val_loss: 0.7019 - val_accuracy: 0.4479
Epoch 6/100
12/17 [====================>.........] - ETA: 0s - loss: 0.5903 - accuracy: 0.6953
Epoch 6: val_loss did not improve from 0.69786
17/17 [==============================] - 0

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_304 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_228 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_152 (Dropout)       (None, 512)               0         
                                                                 
 dense_305 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_229 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_153 (Dropout)       (None, 256)               0         
          

...

22/22 [==============================] - 0s 11ms/step - loss: 0.6601 - accuracy: 0.6365 - val_loss: 0.7195 - val_accuracy: 0.5455
Epoch 7/100
18/22 [=======================>......] - ETA: 0s - loss: 0.6185 - accuracy: 0.6649
Epoch 7: val_loss did not improve from 0.68905
22/22 [==============================] - 0s 11ms/step - loss: 0.6369 - accuracy: 0.6569 - val_loss: 0.7246 - val_accuracy: 0.5455
Epoch 8/100
19/22 [========================>.....] - ETA: 0s - loss: 0.5772 - accuracy: 0.6908
Epoch 8: val_loss did not improve from 0.68905
22/22 [==============================] - 0s 11ms/step - loss: 0.5765 - accuracy: 0.6964 - val_loss: 0.7284 - val_accuracy: 0.5455
Epoch 9/100
19/22 [========================>.....] - ETA: 0s - loss: 0.5583 - accuracy: 0.7089
Epoch 9: val_loss did not improve from 0.68905
22/22 [==============================] - 0s 11ms/step - loss: 0.5516 - accuracy: 0.7153 - val_loss: 0.7329 - val_accuracy: 0.5455
Epoch 10/100
18/22 [=======================>......] - 

Epoch 11/100
23/24 [===========================>..] - ETA: 0s - loss: 0.5181 - accuracy: 0.7269
Epoch 11: val_loss did not improve from 0.70678
24/24 [==============================] - 0s 12ms/step - loss: 0.5192 - accuracy: 0.7270 - val_loss: 0.8280 - val_accuracy: 0.4722
Epoch 12/100
19/24 [======================>.......] - ETA: 0s - loss: 0.5018 - accuracy: 0.7319
Epoch 12: val_loss did not improve from 0.70678
24/24 [==============================] - 0s 11ms/step - loss: 0.5080 - accuracy: 0.7297 - val_loss: 0.8401 - val_accuracy: 0.4722
Epoch 13/100
22/24 [==========================>...] - ETA: 0s - loss: 0.4783 - accuracy: 0.7841
Epoch 13: val_loss did not improve from 0.70678
24/24 [==============================] - 0s 11ms/step - loss: 0.4741 - accuracy: 0.7856 - val_loss: 0.8468 - val_accuracy: 0.4722
Epoch 14/100
19/24 [======================>.......] - ETA: 0s - loss: 0.4839 - accuracy: 0.7615
Epoch 14: val_loss did not improve from 0.70678
24/24 [===========================

Epoch 14/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4329 - accuracy: 0.8105
Epoch 14: val_loss did not improve from 0.68729
17/17 [==============================] - 0s 12ms/step - loss: 0.4306 - accuracy: 0.8123 - val_loss: 0.7326 - val_accuracy: 0.5521
Epoch 15/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4473 - accuracy: 0.8008
Epoch 15: val_loss did not improve from 0.68729
17/17 [==============================] - 0s 13ms/step - loss: 0.4493 - accuracy: 0.7989 - val_loss: 0.7381 - val_accuracy: 0.5521
Epoch 16/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4212 - accuracy: 0.8105
Epoch 16: val_loss did not improve from 0.68729
17/17 [==============================] - 0s 13ms/step - loss: 0.4190 - accuracy: 0.8123 - val_loss: 0.7443 - val_accuracy: 0.5521
Epoch 17/100
17/17 [==============================] - ETA: 0s - loss: 0.4138 - accuracy: 0.8257
Epoch 17: val_loss did not improve from 0.68729
17/17 [===========================


Epoch 16: val_loss did not improve from 0.72310
22/22 [==============================] - 0s 11ms/step - loss: 0.4279 - accuracy: 0.7854 - val_loss: 1.0947 - val_accuracy: 0.1667
Epoch 16: early stopping
4/4 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_Full_text_L_label_layer_2.png
Test Accuracy for Layer 2: 0.3519
Test AUC for Layer 2: 0.4929

Layer 3:
Training period: 01/01/2019 - 31/05/2022
Validation period: 01/06/2022 - 31/12/2022
Testing period: 01/01/2023 - 31/05/2023
Training data: 751 samples
Validation data: 108 samples
Test data: 69 samples
Embeddings shapes - Train: (751, 1536), Val: (108, 1536), Test: (69, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Full text (L_label)
Model: "sequential_83"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_332 (Dense)           (None, 512)               786944    
      

23/24 [===========================>..] - ETA: 0s - loss: 0.4263 - accuracy: 0.8016
Epoch 17: val_loss did not improve from 0.64911
24/24 [==============================] - 0s 11ms/step - loss: 0.4281 - accuracy: 0.8003 - val_loss: 0.6520 - val_accuracy: 0.6667
Epoch 18/100
24/24 [==============================] - ETA: 0s - loss: 0.4240 - accuracy: 0.8123
Epoch 18: val_loss did not improve from 0.64911
24/24 [==============================] - 0s 11ms/step - loss: 0.4240 - accuracy: 0.8123 - val_loss: 0.6544 - val_accuracy: 0.6574
Epoch 19/100
24/24 [==============================] - ETA: 0s - loss: 0.3725 - accuracy: 0.8349
Epoch 19: val_loss did not improve from 0.64911
24/24 [==============================] - 0s 11ms/step - loss: 0.3725 - accuracy: 0.8349 - val_loss: 0.6615 - val_accuracy: 0.6481
Epoch 20/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3542 - accuracy: 0.8388
Epoch 20: val_loss did not improve from 0.64911
24/24 [==============================] - 0s 10m

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import gc
import ast  # For safely parsing string representations of arrays
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Custom callback to plot training history after each epoch
class PlotLearningCallback(Callback):
    def __init__(self, text_col, label_col, layer_num, plot_dir):
        super().__init__()
        self.text_col = text_col
        self.label_col = label_col
        self.layer_num = layer_num
        self.plot_dir = plot_dir
        os.makedirs(plot_dir, exist_ok=True)
        # Keep track of metrics for each epoch
        self.train_acc = []
        self.val_acc = []
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
            
        # Collect metrics
        self.train_acc.append(logs.get('accuracy', logs.get('acc', 0)))
        self.val_acc.append(logs.get('val_accuracy', logs.get('val_acc', 0)))
        self.train_loss.append(logs.get('loss', 0))
        self.val_loss.append(logs.get('val_loss', 0))

class MergedEmbeddingMLPPredictor:
    def __init__(self, csv_path):
        """
        Initialize the stock predictor for climate change news analysis using merged OpenAI embeddings.
        
        Args:
            csv_path: Path to the CSV file containing climate change news data with merged OpenAI embeddings
        """
        self.csv_path = csv_path
        self.data = None
        self.layers = []
        self.results = {}
        
        # Create directories for visualizations
        self.mlp_viz_dir = 'Merged_OpenAI_MLP/visualizations_mlp/COP'
        os.makedirs(self.mlp_viz_dir, exist_ok=True)
        os.makedirs('Merged_OpenAI_MLP/visualizations_summary/COP', exist_ok=True)
            
    def load_data(self):
        """Load and preprocess the CSV data containing climate change news with merged OpenAI embeddings."""
        print("Loading data...")
        self.data = pd.read_csv(self.csv_path)
        
        # Convert date strings to datetime objects
        try:
            # Try pandas' automatic date parsing first with dayfirst=True
            self.data['Publication date'] = pd.to_datetime(self.data['Publication date'], dayfirst=True)
            self.data['Predicting date Short'] = pd.to_datetime(self.data['Predicting date Short'], dayfirst=True)
            self.data['Predicting date Long'] = pd.to_datetime(self.data['Predicting date Long'], dayfirst=True)
        except (ValueError, TypeError):
            print("Automatic date parsing failed. Trying manual conversion...")
            
            # Helper function to handle different date formats
            def parse_date_column(column):
                result = []
                for date_str in column:
                    try:
                        # Try to parse as DD/MM/YYYY
                        date = pd.to_datetime(date_str, format='%d/%m/%Y')
                    except ValueError:
                        try:
                            # Try to parse as YYYY-MM-DD
                            date = pd.to_datetime(date_str, format='%Y-%m-%d')
                        except ValueError:
                            # As a last resort, let pandas guess with dayfirst=True
                            date = pd.to_datetime(date_str, dayfirst=True)
                    result.append(date)
                return pd.Series(result)
            
            self.data['Publication date'] = parse_date_column(self.data['Publication date'])
            self.data['Predicting date Short'] = parse_date_column(self.data['Predicting date Short'])
            self.data['Predicting date Long'] = parse_date_column(self.data['Predicting date Long'])
        
        # Sort by publication date
        self.data = self.data.sort_values('Publication date')
        
        # Parse embedding vectors from string format to numpy arrays
        print("Parsing merged OpenAI embedding vectors from string format...")
        
        # Function to safely convert string representation of array to numpy array
        def parse_embedding(embedding_str):
            if pd.isna(embedding_str):
                # Return zeros array if embedding is missing
                return np.zeros(1536)
            try:
                # Try parsing as a string representation of a list
                embedding_list = ast.literal_eval(embedding_str)
                return np.array(embedding_list, dtype=np.float32)
            except (ValueError, SyntaxError):
                print(f"Error parsing embedding: {embedding_str[:50]}...")
                return np.zeros(1536)
        
        # Convert merged embeddings to numpy arrays
        self.data['Merged_embedding'] = self.data['Merged_embedding_vector'].apply(parse_embedding)
        
        # Verify dimensions
        sample_merged_embedding = self.data['Merged_embedding'].iloc[0]
        print(f"Sample Merged embedding shape: {sample_merged_embedding.shape}")
        
        print(f"Loaded {len(self.data)} climate change news articles spanning from "
              f"{self.data['Publication date'].min().strftime('%d/%m/%Y')} "
              f"to {self.data['Publication date'].max().strftime('%d/%m/%Y')}")
        print(f"Class distribution for short-term prediction: {self.data['S_label'].value_counts().to_dict()}")
        print(f"Class distribution for long-term prediction: {self.data['L_label'].value_counts().to_dict()}")
        
        return self
    
    def define_time_windows(self):
        """Define the time windows for the sliding window approach."""
        # First layer: train (2019-2021), validation (2021-2021), test (2022-2022)
        layer1 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-05-31'),
            'val_start': pd.Timestamp('2021-06-01'),
            'val_end': pd.Timestamp('2021-12-31'),
            'test_start': pd.Timestamp('2022-01-01'),
            'test_end': pd.Timestamp('2022-05-31')
        }
        
        # Second layer: train (2019-2021), validation (2022-2022), test (2022-2022)
        layer2 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2021-12-31'),
            'val_start': pd.Timestamp('2022-01-01'),
            'val_end': pd.Timestamp('2022-05-31'),
            'test_start': pd.Timestamp('2022-06-01'),
            'test_end': pd.Timestamp('2022-12-31')
        }
        
        # Third layer: train (2019-2022), validation (2022-2022), test (2023-2023)
        layer3 = {
            'train_start': pd.Timestamp('2019-01-01'),
            'train_end': pd.Timestamp('2022-05-31'),
            'val_start': pd.Timestamp('2022-06-01'),
            'val_end': pd.Timestamp('2022-12-31'),
            'test_start': pd.Timestamp('2023-01-01'),
            'test_end': pd.Timestamp('2023-05-31')
        }
        
        self.layers = [layer1, layer2, layer3]
        return self
    
    def split_data(self, layer, label_col):
        """
        Split the data into training, validation, and test sets based on the defined time windows.
        Extract OpenAI embeddings for text data.
        
        Args:
            layer: The time window layer
            label_col: The column containing the labels ('S_label' or 'L_label')
            
        Returns:
            train_data, val_data, test_data with embeddings
        """
        train_mask = (self.data['Publication date'] >= layer['train_start']) & (self.data['Publication date'] <= layer['train_end'])
        val_mask = (self.data['Publication date'] > layer['val_start']) & (self.data['Publication date'] <= layer['val_end'])
        test_mask = (self.data['Publication date'] > layer['test_start']) & (self.data['Publication date'] <= layer['test_end'])
        
        train_data = self.data[train_mask]
        val_data = self.data[val_mask]
        test_data = self.data[test_mask]
        
        print(f"Training data: {len(train_data)} samples")
        print(f"Validation data: {len(val_data)} samples")
        print(f"Test data: {len(test_data)} samples")
        
        # Extract embeddings
        X_train = np.stack(train_data['Merged_embedding'].values)
        X_val = np.stack(val_data['Merged_embedding'].values)
        X_test = np.stack(test_data['Merged_embedding'].values)
        
        # Get labels
        y_train = train_data[label_col].values
        y_val = val_data[label_col].values
        y_test = test_data[label_col].values
        
        print(f"Embeddings shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
        
        return (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data
    
    def create_mlp_model(self, input_shape):
        """
        Create a simple MLP model with dense layers (256, 128) for document-level embeddings.
        
        Args:
            input_shape: Shape of the input data (1536,)
            
        Returns:
            Compiled MLP model
        """
        print(f"Creating MLP model with input shape: {input_shape}")
        
        # Create a feed-forward neural network
        model = Sequential([
            # Input layer
            Dense(512, activation='relu', input_shape=input_shape),
            BatchNormalization(),
            Dropout(0.3),
            
            # Hidden layer
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(128, activation='relu'),
            BatchNormalization(),
            
            # Output layer
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.00005),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("MLP model created")
        return model
    
    def plot_final_learning_curves(self, history, label_col, layer_num, visualization_dir):
        """
        Plot final learning curves after training is complete for MLP.
        
        Args:
            history: Training history
            label_col: Label column used
            layer_num: Layer number
            visualization_dir: Directory to save visualizations
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Get history dictionary safely
        history_dict = {}
        if hasattr(history, 'history'):
            history_dict = history.history
        
        # Determine the correct metric names
        acc_metric = 'accuracy' if 'accuracy' in history_dict else 'acc'
        val_acc_metric = 'val_accuracy' if 'val_accuracy' in history_dict else 'val_acc'
        
        # Plot accuracy
        if acc_metric in history_dict and val_acc_metric in history_dict:
            ax1.plot(history_dict[acc_metric], label='Training Accuracy', color='blue')
            ax1.plot(history_dict[val_acc_metric], label='Validation Accuracy', color='orange')
        ax1.set_title(f'Final Accuracy Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(loc='lower right')
        ax1.set_ylim([0, 1])
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Plot loss
        if 'loss' in history_dict and 'val_loss' in history_dict:
            ax2.plot(history_dict['loss'], label='Training Loss', color='blue')
            ax2.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
        ax2.set_title(f'Final Loss Curves: MLP with Merged Embeddings ({label_col}, Layer {layer_num})')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(loc='upper right')
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        filename = f"final_mlp_merged_{label_col}_layer_{layer_num}.png"
        plt.savefig(f"{visualization_dir}/{filename}")
        print(f"Saved learning curves: {filename}")
        plt.close()
    
    def train_and_evaluate_model(self, label_col):
        """
        Train and evaluate an MLP model for merged embeddings and a specific label column.
        
        Args:
            label_col: The label column to use ('S_label' or 'L_label')
        """
        # Store results
        combination_key = f"MLP|Merged|{label_col}"
        self.results[combination_key] = {
            'accuracy': [],
            'auc': [],
            'layer_results': []
        }
        
        print(f"\n{'='*80}")
        print(f"Training MLP model for Merged Embeddings and {label_col}")
        print(f"{'='*80}")
        
        for i, layer in enumerate(self.layers):
            print(f"\nLayer {i+1}:")
            print(f"Training period: {layer['train_start'].strftime('%d/%m/%Y')} - {layer['train_end'].strftime('%d/%m/%Y')}")
            print(f"Validation period: {layer['val_start'].strftime('%d/%m/%Y')} - {layer['val_end'].strftime('%d/%m/%Y')}")
            print(f"Testing period: {layer['test_start'].strftime('%d/%m/%Y')} - {layer['test_end'].strftime('%d/%m/%Y')}")
            
            # Split data and get embeddings
            (X_train, y_train), (X_val, y_val), (X_test, y_test), train_data, val_data, test_data = self.split_data(
                layer, label_col)
            
            # Check if there are enough samples and classes
            if len(X_train) < 10 or len(np.unique(y_train)) < 2 or len(np.unique(y_val)) < 2 or len(np.unique(y_test)) < 2:
                print(f"Skipping layer {i+1} due to insufficient data or class imbalance")
                continue
            
            # Create and train model
            model = self.create_mlp_model((1536,))
            print(f"Created MLP model for Merged Embeddings ({label_col})")
            model.summary()
            
            # Setup callbacks
            plot_callback = PlotLearningCallback('Merged', label_col, i+1, self.mlp_viz_dir)
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            )
            model_checkpoint = ModelCheckpoint(
                filepath=f"{self.mlp_viz_dir}/best_mlp_merged_{label_col}_layer_{i+1}.weights.h5",
                monitor='val_loss',
                save_weights_only=True,
                save_best_only=True,
                verbose=1
            )
            
            # Train model
            print(f"Training MLP model...")
            batch_size = 32  # Larger batch size for embeddings
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=100,  # Increase max epochs, early stopping will prevent overfitting
                batch_size=batch_size,
                callbacks=[early_stopping, model_checkpoint, plot_callback],
                verbose=1
            )
            
            # Evaluate on test set
            y_pred_proba = model.predict(X_test)
            y_pred = (y_pred_proba > 0.5).astype(int)
            
            # Create final learning curve visualization
            self.plot_final_learning_curves(history, label_col, i+1, self.mlp_viz_dir)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            
            self.results[combination_key]['accuracy'].append(accuracy)
            self.results[combination_key]['auc'].append(auc)
            
            print(f"Test Accuracy for Layer {i+1}: {accuracy:.4f}")
            print(f"Test AUC for Layer {i+1}: {auc:.4f}")
            
            # Store layer results for visualization
            layer_result = {
                'layer': i+1,
                'accuracy': accuracy,
                'auc': auc,
                'history': history.history
            }
            
            self.results[combination_key]['layer_results'].append(layer_result)
        
        # Calculate average metrics
        avg_accuracy = np.mean(self.results[combination_key]['accuracy']) if self.results[combination_key]['accuracy'] else 0
        avg_auc = np.mean(self.results[combination_key]['auc']) if self.results[combination_key]['auc'] else 0
        
        print(f"\nAverage Test Accuracy across all layers: {avg_accuracy:.4f}")
        print(f"Average Test AUC across all layers: {avg_auc:.4f}")
        
        self.results[combination_key]['avg_accuracy'] = avg_accuracy
        self.results[combination_key]['avg_auc'] = avg_auc
        
        return self
    
    def run_all_combinations(self):
        """Run the analysis for merged embeddings with both short-term and long-term labels."""
        # Define label columns
        label_cols = ['S_label', 'L_label']
        
        # Run analysis for each label column
        for label_col in label_cols:
            self.train_and_evaluate_model(label_col)
        
        # Print summary
        print("\n" + "="*80)
        print("SUMMARY OF RESULTS (COP)")
        print("="*80)
        
        # Create a summary table for easier comparison
        summary_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            avg_accuracy = results.get('avg_accuracy', 0)
            avg_auc = results.get('avg_auc', 0)
            
            print(f"\nCombination: {model_type} with {text_col} + {label_col}")
            print(f"Average Accuracy: {avg_accuracy:.4f}")
            print(f"Average AUC: {avg_auc:.4f}")
            
            # Print layer-specific results
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                print(f"  Layer {i+1} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
            
            summary_data.append({
                'Combination': f"{text_col} + {label_col}",
                'Avg Accuracy': avg_accuracy,
                'Avg AUC': avg_auc,
                'Label': label_col
            })
        
        # Create summary visualizations
        self.create_summary_visualization(summary_data)
        
        # Clean up memory
        gc.collect()
        
        return self
    
    def create_summary_visualization(self, summary_data):
        """Create a summary visualization comparing model combinations."""
        if not summary_data:
            print("No data available for summary visualization")
            return
        
        # Create a DataFrame for easier plotting
        df = pd.DataFrame(summary_data)
        
        # Convert metrics to float for plotting
        df['Avg Accuracy'] = df['Avg Accuracy'].astype(float)
        df['Avg AUC'] = df['Avg AUC'].astype(float)
        
        # Performance comparison for MLP with merged embeddings
        plt.figure(figsize=(10, 6))
        
        # Plot accuracy and AUC bars
        x = np.arange(len(df))
        width = 0.35
        
        plt.bar(x - width/2, df['Avg Accuracy'], width, label='Average Accuracy', color='skyblue')
        plt.bar(x + width/2, df['Avg AUC'], width, label='Average AUC', color='salmon')
        
        plt.xlabel('Model Combination')
        plt.ylabel('Score')
        plt.title('Performance Comparison of MLP Models with Merged OpenAI Embeddings')
        labels = [f"Merged + {row['Label']}" for _, row in df.iterrows()]
        plt.xticks(x, labels, rotation=0)
        plt.legend()
        
        # Add value labels on top of bars
        for i, v in enumerate(df['Avg Accuracy']):
            plt.text(i - width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        for i, v in enumerate(df['Avg AUC']):
            plt.text(i + width/2, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=9)
        
        plt.ylim(0, max(df['Avg Accuracy'].max(), df['Avg AUC'].max()) + 0.1)
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/COP', "mlp_merged_performance_comparison.png")
        plt.savefig(save_path)
        print(f"MLP summary comparison visualization saved as: {save_path}")
        plt.close()
        
        # Layer-specific performance visualizations
        self.create_layer_performance_visualizations()
    
    def create_layer_performance_visualizations(self):
        """Create visualizations showing performance across different layers for MLP models."""
        # Prepare data for visualization
        layer_data = []
        
        for combination, results in self.results.items():
            model_type, text_col, label_col = combination.split('|')
            
            for i, accuracy in enumerate(results.get('accuracy', [])):
                auc = results.get('auc', [])[i]
                layer_info = {
                    'Text': text_col,
                    'Label': label_col,
                    'Layer': f"Layer {i+1}",
                    'Layer_num': i+1,
                    'Accuracy': accuracy,
                    'AUC': auc,
                    'Combination': f"{text_col} + {label_col}"
                }
                
                layer_data.append(layer_info)
        
        if not layer_data:
            print("No layer data available")
            return
        
        # Create DataFrame
        df = pd.DataFrame(layer_data)
        
        # Create visualization - Accuracy by layer
        plt.figure(figsize=(10, 8))
        
        # 1. Accuracy by combination and layer
        plt.subplot(2, 1, 1)
        sns.barplot(x='Combination', y='Accuracy', hue='Layer', data=df)
        plt.title('MLP Accuracy by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['Accuracy'].max() + 0.05))
        plt.xticks(rotation=0)
        plt.tight_layout()
        
        # 2. AUC by combination and layer
        plt.subplot(2, 1, 2)
        sns.barplot(x='Combination', y='AUC', hue='Layer', data=df)
        plt.title('MLP AUC by Model Combination and Layer with Merged OpenAI Embeddings')
        plt.ylim(0, max(0.8, df['AUC'].max() + 0.05))
        plt.xticks(rotation=0)
        
        plt.tight_layout()
        
        # Save the visualization
        save_path = os.path.join('Merged_OpenAI_MLP/visualizations_summary/COP', "mlp_merged_layer_performance.png")
        plt.savefig(save_path)
        print(f"MLP layer performance visualization saved as: {save_path}")
        plt.close()


# Main execution
if __name__ == "__main__":
    # Ensure visualization directories exist
    for directory in ['Merged_OpenAI_MLP/visualizations_mlp/COP', 'Merged_OpenAI_MLP/visualizations_summary/COP']:
        os.makedirs(directory, exist_ok=True)
    
    csv_path = 'E:/User/Documents/Documents (UK)/Cardiff (PhD)/First Year/climate change/US_news/SP500_semantic/us_news_semantics_COP_completed_openai.csv'
    
    # Initialize the predictor with merged OpenAI embeddings
    predictor = MergedEmbeddingMLPPredictor(csv_path)
    
    # Run the complete analysis
    predictor.load_data().define_time_windows().run_all_combinations()
    
    print("\nAnalysis complete! Results saved to 'Merged_OpenAI_MLP' directory.")

Loading data...
Automatic date parsing failed. Trying manual conversion...
Parsing merged OpenAI embedding vectors from string format...
Sample Merged embedding shape: (1536,)
Loaded 929 climate change news articles spanning from 02/01/2019 to 07/05/2023
Class distribution for short-term prediction: {0: 469, 1: 460}
Class distribution for long-term prediction: {1: 504, 0: 425}

Training MLP model for Merged Embeddings and S_label

Layer 1:
Training period: 01/01/2019 - 31/05/2021
Validation period: 01/06/2021 - 31/12/2021
Testing period: 01/01/2022 - 31/05/2022
Training data: 522 samples
Validation data: 163 samples
Test data: 66 samples
Embeddings shapes - Train: (522, 1536), Val: (163, 1536), Test: (66, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (S_label)
Model: "sequential_84"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_336 

 dense_340 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_255 (B  (None, 512)               2048      
 atchNormalization)                                              
                                                                 
 dropout_170 (Dropout)       (None, 512)               0         
                                                                 
 dense_341 (Dense)           (None, 256)               131328    
                                                                 
 batch_normalization_256 (B  (None, 256)               1024      
 atchNormalization)                                              
                                                                 
 dropout_171 (Dropout)       (None, 256)               0         
                                                                 
 dense_342 (Dense)           (None, 128)               32896     
          

                                                                 
 dense_346 (Dense)           (None, 128)               32896     
                                                                 
 batch_normalization_260 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 dense_347 (Dense)           (None, 1)                 129       
                                                                 
Total params: 954881 (3.64 MB)
Trainable params: 953089 (3.64 MB)
Non-trainable params: 1792 (7.00 KB)
_________________________________________________________________
Training MLP model...
Epoch 1/100
21/24 [=========================>....] - ETA: 0s - loss: 0.8978 - accuracy: 0.4926
Epoch 1: val_loss improved from inf to 0.70019, saving model to Merged_OpenAI_MLP/visualizations_mlp/COP\best_mlp_merged_S_label_layer_3.weights.h5
24/24 [==============================]

17/17 [==============================] - 2s 26ms/step - loss: 0.9361 - accuracy: 0.4770 - val_loss: 0.6959 - val_accuracy: 0.4479
Epoch 2/100
17/17 [==============================] - ETA: 0s - loss: 0.8716 - accuracy: 0.4828
Epoch 2: val_loss did not improve from 0.69588
17/17 [==============================] - 0s 13ms/step - loss: 0.8716 - accuracy: 0.4828 - val_loss: 0.6990 - val_accuracy: 0.4479
Epoch 3/100
13/17 [=====================>........] - ETA: 0s - loss: 0.7809 - accuracy: 0.5697
Epoch 3: val_loss did not improve from 0.69588
17/17 [==============================] - 0s 12ms/step - loss: 0.7787 - accuracy: 0.5690 - val_loss: 0.7014 - val_accuracy: 0.4479
Epoch 4/100
17/17 [==============================] - ETA: 0s - loss: 0.7415 - accuracy: 0.5785
Epoch 4: val_loss did not improve from 0.69588
17/17 [==============================] - 0s 12ms/step - loss: 0.7415 - accuracy: 0.5785 - val_loss: 0.7027 - val_accuracy: 0.4479
Epoch 5/100
13/17 [=====================>........] - E

Epoch 5/100
22/22 [==============================] - ETA: 0s - loss: 0.6428 - accuracy: 0.6526
Epoch 5: val_loss improved from 0.64153 to 0.63253, saving model to Merged_OpenAI_MLP/visualizations_mlp/COP\best_mlp_merged_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 14ms/step - loss: 0.6428 - accuracy: 0.6526 - val_loss: 0.6325 - val_accuracy: 0.8333
Epoch 6/100
21/22 [===========================>..] - ETA: 0s - loss: 0.6463 - accuracy: 0.6458
Epoch 6: val_loss improved from 0.63253 to 0.62551, saving model to Merged_OpenAI_MLP/visualizations_mlp/COP\best_mlp_merged_L_label_layer_2.weights.h5
22/22 [==============================] - 0s 14ms/step - loss: 0.6440 - accuracy: 0.6467 - val_loss: 0.6255 - val_accuracy: 0.8333
Epoch 7/100
21/22 [===========================>..] - ETA: 0s - loss: 0.6014 - accuracy: 0.6786
Epoch 7: val_loss improved from 0.62551 to 0.61753, saving model to Merged_OpenAI_MLP/visualizations_mlp/COP\best_mlp_merged_L_label_layer_2.weights.h5

Epoch 30: early stopping
4/4 [==============================] - 0s 3ms/step
Saved learning curves: final_mlp_merged_L_label_layer_2.png
Test Accuracy for Layer 2: 0.6667
Test AUC for Layer 2: 0.5256

Layer 3:
Training period: 01/01/2019 - 31/05/2022
Validation period: 01/06/2022 - 31/12/2022
Testing period: 01/01/2023 - 31/05/2023
Training data: 751 samples
Validation data: 108 samples
Test data: 69 samples
Embeddings shapes - Train: (751, 1536), Val: (108, 1536), Test: (69, 1536)
Creating MLP model with input shape: (1536,)
MLP model created
Created MLP model for Merged Embeddings (L_label)
Model: "sequential_89"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_356 (Dense)           (None, 512)               786944    
                                                                 
 batch_normalization_267 (B  (None, 512)               2048      
 atchNormalization)                             

Epoch 19/100
22/24 [==========================>...] - ETA: 0s - loss: 0.3742 - accuracy: 0.8523
Epoch 19: val_loss improved from 0.64683 to 0.64502, saving model to Merged_OpenAI_MLP/visualizations_mlp/COP\best_mlp_merged_L_label_layer_3.weights.h5
24/24 [==============================] - 0s 14ms/step - loss: 0.3762 - accuracy: 0.8509 - val_loss: 0.6450 - val_accuracy: 0.6944
Epoch 20/100
23/24 [===========================>..] - ETA: 0s - loss: 0.3582 - accuracy: 0.8587
Epoch 20: val_loss did not improve from 0.64502
24/24 [==============================] - 0s 11ms/step - loss: 0.3552 - accuracy: 0.8615 - val_loss: 0.6482 - val_accuracy: 0.6852
Epoch 21/100
19/24 [======================>.......] - ETA: 0s - loss: 0.3410 - accuracy: 0.8668
Epoch 21: val_loss did not improve from 0.64502
24/24 [==============================] - 0s 10ms/step - loss: 0.3402 - accuracy: 0.8628 - val_loss: 0.6497 - val_accuracy: 0.6759
Epoch 22/100
24/24 [==============================] - ETA: 0s - loss: 0.3